In [1]:
import pandas as pd
import geopandas as gp
import numpy as np 
import os
import fiona
from statistics import mean, median
import string
import sys

# Oregon 2020

## VEST Documentation

Election results from the Oregon Secretary of State via OpenElections (https://github.com/openelections/openelections-data-or/). Results from Polk, Tillamook, Wallowa, and Wasco counties were corrected with the respective county canvass reports.

Precinct shapefiles for the following counties were provided by the respective county governments: Benton, Clatsop, Columbia, Coos, Crook, Curry, Deschutes, Douglas, Harney, Hood River, Jackson, Josephine, Klamath, Lane, Linn, Malheur, Morrow, Multnomah, Polk, Sherman, Tillamook, Umatilla, Union, Wallowa, Wasco, Yamhill.

Precinct shapefiles for the following counties were provided by the Oregon Secretary of State: Jefferson, Lane, Marion. Several gaps in the Marion County shapefile were assigned based on the Precinct Split Summary Report.

Precinct shapefiles for Clackamas and Washington were provided by the Metro Regional Government Data Resource Center.

Precinct boundaries for Grant County and Wheeler County are defined by school district boundaries. Precinct shapefiles were produced to match PDF maps provided by the respective counties using the 2016 Oregon Education Boundaries shapefile obtained from the Oregon Spatial Data Library.

Precinct shapefiles for the following counties were produced based on PDF maps provided by the respective counties: Baker, Gilliam, Lake, Lincoln. Boundaries defined by the PLSS grid were produced with the Oregon Public Land Survey Quarter-Quarter Reference Grid shapefile obtained from the Oregon Spatial Data Library. Municipal boundaries were produced from the Oregon Department of Transportion 2016 City Limits shapefile obtained from the Oregon Spatial Data Library. Street boundaries were produced from the U.S. Census Bureau census block shapefiles.

PDF maps obtained from Lake County are approximate precinct boundaries drawn on the PLSS grid for county precincts and on the street map for Lakeview city precincts. These boundaries were adjusted to match address range assignments in the Lake County Address Library Report which serves as the legal definition of the precincts according to the Lake County Clerk. Precinct divisions across roads that span multiple rural precincts are generally defined by zip codes. Address ranges were further identified based on the Lake County tax lot parcel viewer.

Rural precinct boundaries in the shapefiles provided by Coos County and Union County were revised to match the updated PLSS cadastral grid in the Oregon GIS Framework obtained from the Oregon Spatial Data Library. 

Municipal precinct boundaries in the following counties were edited to match city limits in effect for the November 2020 general election: Coos, Deschutes, Douglas, Jefferson, Malheur, Marion, Tillamook, Umatilla, Washington, Yamhill.

Precinct numbers in the Columbia County, Josephine County, and Wallowa County shapefiles were edited to match the Oregon Secretary of State voter file.

G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRERTRU - Donald J. Trump (Republican Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
G20PREGHAW - Howie Hawkins (Pacific Green Party)  
G20PREPHUN - Dario Hunter (Progressive Party)  
G20PREOWRI - Write-in Votes  

G20USSDMER - Jeff Merkley (Democratic Party, Independent Party and Working Family Party (fusion candidate))  
G20USSRPER - Jo Rae Perkins (Republican Party)  
G20USSLDYE - Gary Dye (Libertarian Party)  
G20USSGTAH - Ibrahim A. Taher (Pacific Green Party and Progressive Party (fusion candidate))  
G20USSOWRI - Write-in Votes  

G20ATGDROS - Ellen Rosenblum (Democratic Party, Independent Party and Working Family Party (fusion candidate))  
G20ATGRCRO - Michael Cross (Republican Party)  
G20ATGLHED - Lars D. H. Hedbor (Libertarian Party)  
G20ATGOWRI - Write-in Votes  

G20SOSDFAG - Shemia Fagan (Democratic Party and Working Family Party (fusion candidate))  
G20SOSRTHA - Kim Thatcher (Republican Party and Independent Party (fusion candidate))  
G20SOSLMAR - Kyle Markley (Libertarian Party)  
G20SOSGPAR - Nathalie Paravicini (Pacific Green Party and Progressive Party (fusion candidate))  
G20SOSOWRI - Write-in Votes  

G20TREDREA - Tobias Read (Democratic Party and Working Family Party (fusion candidate))  
G20TRERGUD - Jeff Gudman (Republican Party)  
G20TREIHEN - Chris Henry (Independent Party, Pacific Green Party and Progressive Party (fusion candidate))  
G20TRECMAR - Michael P. Marsh (Constitution Party)  
G20TREOWRI - Write-in Votes  

## Load VEST File

In [2]:
vest_or_20 = gp.read_file("./raw-from-source/VEST/or_2020/or_2020.shp")

In [3]:
data_columns = [x for x in vest_or_20.columns if "G20" in x]

## Load OR Votes

In [4]:
file_list = os.listdir("./raw-from-source/Election_Results/2020/counties")
li = []
for file in file_list:
    #print(file)
    if "general" in file:
        #print(file)
        file_ref = "./raw-from-source/Election_Results/2020/counties/"+file
        file_prev = pd.read_csv(file_ref)
        if 'precincts' in file_prev.columns:
            file_prev['precinct'] = file_prev['precincts']
     
        file_prev = file_prev[['county', 'precinct', 'office', 'district', 'party', 'candidate','votes']]
        li.append(file_prev)
election_results = pd.concat(li, axis=0, ignore_index=True)
display(election_results)

,county,precinct,office,district,party,candidate,votes
0,Linn,Precinct 001,President,NaN,REP,Donald J. Trump,208.0
1,Linn,Precinct 001,President,NaN,DEM,Joseph R. Biden,428.0
2,Linn,Precinct 001,President,NaN,LBT,Jo Jorgensen,11.0
3,Linn,Precinct 001,President,NaN,PGP,Howie Hawkins,8.0
4,Linn,Precinct 001,President,NaN,PRO,Dario Hunter,7.0
...,...,...,...,...,...,...,...
81840,Malheur,20 - City of Jordan Valley,State House,60.0,NaN,Under Votes,1.0
81841,Malheur,21 - Rural Jordan Valley,State House,60.0,NaN,Under Votes,5.0
81842,Malheur,22 - Arock,State House,60.0,NaN,Under Votes,0.0
81843,Malheur,23 - McDermitt,State House,60.0,NaN,Under Votes,3.0


In [5]:
election_results["office"].unique()

array(['President', 'U.S. Senate', 'U.S. House', 'Secretary of State',
       'State Treasurer', 'Attorney General', 'State House',
       'County Commissioner, Position 2',
       'County Commissioner, Position 3',
       'Judge of the Supreme Court, Position 4',
       'Judge of the Court of Appeals, Position 9',
       'Judge of the Circuit Court, 23rd District, Position 4',
       'County Treasurer',
       'Linn Soil & Water Conservation District, Director, Zone 2',
       'Linn Soil & Water Conservation District, Director, Zone 4',
       'Linn Soil & Water Conservation District, Director, Zone 5',
       'Linn Soil & Water Conservation District, Director, At Large',
       'Mayor - City of Albany', 'City of Albany - Councilor, Ward IB',
       '22-183 Four Year Law Enforcement Local Option Tax Levy',
       '107 Limiting/identifing political campaign contributions',
       '108 Increase cigarette and cigar taxes',
       '109 Allows manufacture, delivery, admin. of psilocybinVot

In [6]:
office_list = ['President', 'U.S. Senate','Secretary of State',
       'State Treasurer', 'Attorney General']
election_results = election_results[election_results["office"].isin(office_list)]


In [7]:
election_results["pivot_col"] = election_results["county"] +"-" + election_results["precinct"].astype(str)

In [8]:
election_results["candidate"].unique()

drop_cand_list = ['Over Votes', 'Under Votes','undervote','over']

election_results = election_results[~election_results["candidate"].isin(drop_cand_list)]

In [9]:
election_results["cand_detailed"] = election_results["office"]+"-"+election_results["candidate"]

In [10]:
election_results["cand_detailed"].unique()

array(['President-Donald J. Trump', 'President-Joseph R. Biden',
       'President-Jo Jorgensen', 'President-Howie Hawkins',
       'President-Dario Hunter', 'President-Write-in',
       'U.S. Senate-Jo Rae Perkins', 'U.S. Senate-Jeff Merkley',
       'U.S. Senate-Ibrahim A Taher', 'U.S. Senate-Gary Dye',
       'U.S. Senate-Write-in', 'Secretary of State-Nathalie Paravicini',
       'Secretary of State-Kyle Markley',
       'Secretary of State-Kim Thatcher',
       'Secretary of State-Shemia Fagan', 'Secretary of State-Write-in',
       'State Treasurer-Michael P Marsh', 'State Treasurer-Tobias Read',
       'State Treasurer-Chris Henry', 'State Treasurer-Jeff Gudman',
       'State Treasurer-Write-in', 'Attorney General-Ellen Rosenblum',
       'Attorney General-Lars D H Hedbor',
       'Attorney General-Michael Cross', 'Attorney General-Write-in',
       'Attorney General-Write-ins', 'President-Write-ins',
       'Secretary of State-Write-ins', 'State Treasurer-Write-ins',
       'U

G20ATGDROS - Ellen Rosenblum (Democratic Party, Independent Party and Working Family Party (fusion candidate))
G20ATGRCRO - Michael Cross (Republican Party)
G20ATGLHED - Lars D. H. Hedbor (Libertarian Party)
G20ATGOWRI - Write-in Votes

In [11]:
cand_name_updates_dict = {'President-Donald J. Trump':"G20PRERTRU",
 'President-Joseph R. Biden':"G20PREDBID",
 'President-Jo Jorgensen':"G20PRELJOR", 
 'President-Howie Hawkins':"G20PREGHAW",
 'President-Dario Hunter':"G20PREPHUN", 
 'President-Write-in':"G20PREOWRI",
 'President-Jo Jorgensen /Jeremy (Spike) Cohen':"G20PRELJOR",
 'President-Jo Jorgensen / JeremyjSpike) Cohen':"G20PRELJOR",
 'President-Joseph R Biden / Kamala D Harris':"G20PREDBID",
 'President-Jo Jorgensen / Jeremy (Spike) Cohen':"G20PRELJOR",
 'President-Jo Jorgensen / Jeremy. (Spike) Cohen':"G20PRELJOR",
 'President-Jo Jorgensen / Jeremy Spike) Cohen':"G20PRELJOR",
 'President-Write-ins':"G20PREOWRI",
 'President-Howie Hawkins / Angela Walker':"G20PREGHAW",
 'President-Ballots Cast':"drop",
 'President-Writein':"G20PREOWRI",
 'President-Write-Ins':"G20PREOWRI",
 'President-Write-In':"G20PREOWRI",
 'President-Joseph R. Biden / Kamala D Harris':"G20PREDBID",

 
 'U.S. Senate-Jo Rae Perkins':"G20USSRPER",
 'U.S. Senate-Jeff Merkley':"G20USSDMER",
 'U.S. Senate-Ibrahim A Taher':"G20USSGTAH",
 'U.S. Senate-Gary Dye':"G20USSLDYE",
 'U.S. Senate-Write-in':"G20USSOWRI",
 'U.S. Senate-Write-ins':"G20USSOWRI",
 'U.S. Senate-Ibrahim A. Taher':"G20USSGTAH",
 'U.S. Senate-Ibrahim Tahir':"G20USSGTAH",
 'U.S. Senate-Gary.Dye':"G20USSLDYE",
 'U.S. Senate-Ibrahim Taher':"G20USSGTAH",
 'U.S. Senate-Ibraham Taher':"G20USSGTAH",
 'U.S. Senate-Write-In':"G20USSOWRI",
 'U.S. Senate-Write-Ins':"G20USSOWRI",
 
 'Secretary of State-Nathalie Paravicini':"G20SOSGPAR",
 'Secretary of State-Natalie Paravicini':"G20SOSGPAR",
 'Secretary of State-Nathalie Paravacini':"G20SOSGPAR",
 'Secretary of State-Nathalie Paracivini':"G20SOSGPAR",
 'Secretary of State-Kyle Markey':"G20SOSLMAR",
 'Secretary of State-Kyle Markley':"G20SOSLMAR",
 'Secretary of State-Kim Thatcher':"G20SOSRTHA",
 'Secretary of State-Shemia Fagan':"G20SOSDFAG",
 'Secretary of State-Write-in':"G20SOSOWRI",
 'Secretary of State-Write-Ins':"G20SOSOWRI",
 'Secretary of State-Write-In':"G20SOSOWRI", 
 'Secretary of State-Write-ins':"G20SOSOWRI",
 'Secretary of State-Writein':"G20SOSOWRI",
 
 'State Treasurer-Jeff Gudman':"G20TRERGUD",
 'State Treasurer-Michael P Marsh':"G20TRECMAR", 
 'State Treasurer-Michael Marsh':"G20TRECMAR",
 'State Treasurer-Michael P. Marsh':"G20TRECMAR",
 'State Treasurer-Michael P. Marsch':"G20TRECMAR",
 'State Treasurer-Tobias Read':"G20TREDREA",
 'State Treasurer-Tobias Reed':"G20TREDREA",
 'State Treasurer-Writein':"G20TREOWRI",
 'State Treasurer-Write-ins':"G20TREOWRI",
 'State Treasurer-Write-Ins':"G20TREOWRI",
 'State Treasurer-Write-In':"G20TREOWRI",
 'State Treasurer-Write-in':"G20TREOWRI",
 'State Treasurer-IND Chris Henry':"G20TREIHEN",
 'State Treasurer-Chris Henry':"G20TREIHEN",
 'State Treasurer-Chris Hen':"G20TREIHEN",
 
 'Attorney General-Ellen Rosenblum':"G20ATGDROS",
 'Attorney General-Ellen Roseblum':"G20ATGDROS",
 'Attorney General-Lars D H Hedbor':"G20ATGLHED",
 'Attorney General-Lars D.H. Hedbor':"G20ATGLHED",
 'Attorney General-Lars D. H. Hedbor':"G20ATGLHED",
 'Attorney General-Lars Hedbor':"G20ATGLHED",
 'Attorney General-Michael Cross':"G20ATGRCRO", 
 'Attorney General-Write-in':"G20ATGOWRI",
 'Attorney General-Write-ins':"G20ATGOWRI",
 'Attorney General-Writein':"G20ATGOWRI",
 'Attorney General-Write-Ins':"G20ATGOWRI",
 'Attorney General-Write-In':"G20ATGOWRI"
}

In [12]:
election_results["cand_detailed"] = election_results["cand_detailed"].map(cand_name_updates_dict).fillna(election_results["cand_detailed"])

In [13]:
election_results = election_results[election_results["cand_detailed"]!="drop"]

In [14]:
pivoted_results = pd.pivot_table(election_results,index="pivot_col",values=["votes"],aggfunc=sum,columns="cand_detailed")

In [15]:
pivoted_results.reset_index(inplace=True,drop=False)
pivoted_results.columns = pivoted_results.columns.droplevel(0)
pivoted_results.rename(columns={"":"unique_ID"},inplace=True)

In [16]:
pivoted_results.columns

Index(['unique_ID', 'G20ATGDROS', 'G20ATGLHED', 'G20ATGOWRI', 'G20ATGRCRO',
       'G20PREDBID', 'G20PREGHAW', 'G20PRELJOR', 'G20PREOWRI', 'G20PREPHUN',
       'G20PRERTRU', 'G20SOSDFAG', 'G20SOSGPAR', 'G20SOSLMAR', 'G20SOSOWRI',
       'G20SOSRTHA', 'G20TRECMAR', 'G20TREDREA', 'G20TREIHEN', 'G20TREOWRI',
       'G20TRERGUD', 'G20USSDMER', 'G20USSGTAH', 'G20USSLDYE', 'G20USSOWRI',
       'G20USSRPER'],
      dtype='object', name='cand_detailed')

In [17]:
pivoted_results = pivoted_results[pivoted_results["unique_ID"]!="Curry-nan"]

## Statewide Check

In [18]:
def statewide_totals_check(partner_df,source_df,column_list):
    """Compares the totals of two election result dataframes at the statewide total level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
 
    Returns:
      Nothing, only prints out an analysis
    """
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\tVEST: "+str(partner_df[race].sum())+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(partner_df[race].sum()))

In [19]:
statewide_totals_check(vest_or_20,pivoted_results,data_columns)

***Statewide Totals Check***
G20PREDBID is equal 	VEST / RDH: 1340383
G20PRERTRU is equal 	VEST / RDH: 958448
G20PRELJOR has a difference of 20.0 votes
	VEST: 41582 votes
	SOURCES: 41562.0 votes
G20PREGHAW has a difference of 8.0 votes
	VEST: 11831 votes
	SOURCES: 11823.0 votes
G20PREPHUN is equal 	VEST / RDH: 4988
G20PREOWRI is equal 	VEST / RDH: 17089
G20USSDMER is equal 	VEST / RDH: 1321047
G20USSRPER is equal 	VEST / RDH: 912814
G20USSLDYE is equal 	VEST / RDH: 42747
G20USSGTAH is equal 	VEST / RDH: 42239
G20USSOWRI is equal 	VEST / RDH: 2402
G20ATGDROS is equal 	VEST / RDH: 1264716
G20ATGRCRO is equal 	VEST / RDH: 934357
G20ATGLHED is equal 	VEST / RDH: 52087
G20ATGOWRI is equal 	VEST / RDH: 8490
G20SOSDFAG is equal 	VEST / RDH: 1146370
G20SOSRTHA is equal 	VEST / RDH: 984597
G20SOSLMAR is equal 	VEST / RDH: 62985
G20SOSGPAR is equal 	VEST / RDH: 82211
G20SOSOWRI is equal 	VEST / RDH: 2340
G20TREDREA is equal 	VEST / RDH: 1166703
G20TRERGUD is equal 	VEST / RDH: 936916
G20TREIHEN 

## County-by-County Check

In [20]:
pivoted_results["COUNTY"] = pivoted_results["unique_ID"].apply(lambda x:x.split("-")[0])

In [21]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    """Compares the totals of two election result dataframes at the county level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
      county_col: String of the column name that contains county information
      full_print: Boolean specifying whether to print out everything, including counties w/ similarities

    Returns:
      Nothing, only prints out an analysis
    """
    
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
    if (len(diff_counties)>0):
        print()
        print(diff_counties)

In [22]:
county_totals_check(vest_or_20,pivoted_results,data_columns,"COUNTY",full_print=False)

***Countywide Totals Check***

G20PREDBID is equal across all counties
G20PRERTRU is equal across all counties
G20PRELJOR contains differences in these counties:
	Baker has a difference of 20.0 votes
		VEST: 144 votes
		SOURCES: 124.0 votes
G20PREGHAW contains differences in these counties:
	Grant has a difference of 8.0 votes
		VEST: 12 votes
		SOURCES: 4.0 votes
G20PREPHUN is equal across all counties
G20PREOWRI is equal across all counties
G20USSDMER is equal across all counties
G20USSRPER is equal across all counties
G20USSLDYE is equal across all counties
G20USSGTAH is equal across all counties
G20USSOWRI is equal across all counties
G20ATGDROS is equal across all counties
G20ATGRCRO is equal across all counties
G20ATGLHED is equal across all counties
G20ATGOWRI is equal across all counties
G20SOSDFAG is equal across all counties
G20SOSRTHA is equal across all counties
G20SOSLMAR is equal across all counties
G20SOSGPAR is equal across all counties
G20SOSOWRI is equal across all co

In [23]:
vest_or_20.groupby(["COUNTY"]).sum().to_csv("./vest_totals.csv")

## Precinct-by-Precinct Check

In [24]:
vest_or_20["unique_ID"]=vest_or_20["COUNTY"]+"-"+vest_or_20["NAME"]+" "+vest_or_20["PRECINCT"]
vest_or_20["unique_ID"] = vest_or_20["unique_ID"].str.upper()

In [25]:
pivoted_results["unique_ID"] = pivoted_results["unique_ID"].str.upper()

In [26]:
vest_or_20["unique_ID"].value_counts(dropna=False)
pivoted_results["unique_ID"].value_counts(dropna=False)

WASHINGTON-PRECINCT 408       1
CLACKAMAS-031                 1
LANE-PRECINCT 730             1
COLUMBIA-44 RURAL VERNONIA    1
UNION-11-IMB/SUM RURAL        1
                             ..
DESCHUTES-PRECINCT 24         1
KLAMATH-PRECINCT 13           1
LAKE-SILVER LAKE              1
LINN-PRECINCT 076             1
MARION-321                    1
Name: unique_ID, Length: 1326, dtype: int64

In [27]:
join_attempt_one = pd.merge(vest_or_20,pivoted_results,on="unique_ID",how="outer",indicator=True)
join_attempt_one["_merge"].value_counts()

left_only     1331
right_only    1326
both             0
Name: _merge, dtype: int64

In [28]:

join_attempt_one[join_attempt_one["_merge"]=="left_only"].to_csv("./vest_only.csv")
join_attempt_one[join_attempt_one["_merge"]=="right_only"].to_csv("./pivoted.csv")

In [30]:
source_vest_id_changes_dict = {'BAKER-BAKER #1': 'BAKER-BAKER CITY 1 01', 'BAKER-BAKER #2': 'BAKER-BAKER CITY 2 02', 'BAKER-BAKER #3': 'BAKER-BAKER CITY 3 03', 'BAKER-BAKER #4': 'BAKER-BAKER CITY 4 04', 'BAKER-BAKER #5': 'BAKER-BAKER CITY 5 05', 'BAKER-BAKER COUNTRY': 'BAKER-BAKER 13', 'BAKER-DURKEE': 'BAKER-DURKEE 14', 'BAKER-EAGLE VALLEY': 'BAKER-EAGLE VALLEY 15', 'BAKER-HAINES': 'BAKER-HAINES 17', 'BAKER-HEREFORD': 'BAKER-HEREFORD 18', 'BAKER-HUNTINGTON': 'BAKER-HUNTINGTON 19', 'BAKER-IRONDYKE': 'BAKER-IRONDYKE 20', 'BAKER-KEATING': 'BAKER-KEATING 21', 'BAKER-PINE VALLEY': 'BAKER-PINE VALLEY 22', 'BAKER-POCA-WING': 'BAKER-POCO-WING 24', 'BAKER-SUMPTER': 'BAKER-SUMPTER 25', 'BAKER-UNITY': 'BAKER-UNITY 26', 'BENTON-01 - CORVALLIS ONE C4S8H16': 'BENTON-CORVALLIS 1 01', 'BENTON-02 - CORVALLIS TWO C4S8H16': 'BENTON-CORVALLIS 2 02', 'BENTON-03 - CORVALLIS THREE C4S8H16': 'BENTON-CORVALLIS 3 03', 'BENTON-04 - CORVALLIS FOUR C4S8H16': 'BENTON-CORVALLIS 4 04', 'BENTON-05 - CORVALLIS FIVE C4S8H16': 'BENTON-CORVALLIS 5 05', 'BENTON-06 - CORVALLIS SIX C4S8H16': 'BENTON-CORVALLIS 6 06', 'BENTON-07 - CORVALLIS SEVEN C4S8H16': 'BENTON-CORVALLIS 7 07', 'BENTON-08 - CORVALLIS EIGHT C4S8H16': 'BENTON-CORVALLIS 8 08', 'BENTON-09 - CORVALLIS NINE C4S8H16': 'BENTON-CORVALLIS 9 09', 'BENTON-10 - PHILOMATH C4S8H16': 'BENTON-PHILOMATH CITY 10', 'BENTON-11 - N ALBANY C5S8H15': 'BENTON-N ALBANY 11', 'BENTON-12 - ADAIR VILLAGE C5S12H23': 'BENTON-ADAIR VILLAGE 12', 'BENTON-13 - MONROE C4S12H23': 'BENTON-MONROE CITY 13', 'BENTON-14 - ALSEA SCHOOL C4S12H23': 'BENTON-ALSEA SCHOOL 14', 'BENTON-15 - NORTHWEST C4S12H23': 'BENTON-NORTHWEST 15', 'BENTON-16 - CORVALLIS RURAL C4S8H16': 'BENTON-CORVALLIS RURAL 16', 'BENTON-17 - NORTH 509J C4S12H23': 'BENTON-NORTH 17', 'BENTON-18 - ALBANY RURAL C5S8H15': 'BENTON-ALBANY RURAL 18', 'BENTON-19 - SOUTH 509J C4S12H23': 'BENTON-SOUTH 19', 'BENTON-20 - MONROE RURAL C4S12H23': 'BENTON-MONROE RURAL 20', 'BENTON-21 - PHILOMATH RURAL C4S8H16': 'BENTON-PHILOMATH RURAL 21', 'BENTON-22 - ADAIR RURAL C5S12H23': 'BENTON-ADAIR RURAL 22', 'BENTON-23 - HIGHWAY 20 C4S8H15': 'BENTON-HIGHWAY 20 23', 'CLACKAMAS-002': 'CLACKAMAS-002 002', 'CLACKAMAS-003': 'CLACKAMAS-003 003', 'CLACKAMAS-004': 'CLACKAMAS-004 004', 'CLACKAMAS-005': 'CLACKAMAS-005 005', 'CLACKAMAS-006': 'CLACKAMAS-006 006', 'CLACKAMAS-007 & 010': 'CLACKAMAS-007/010 007/010', 'CLACKAMAS-031': 'CLACKAMAS-031 031', 'CLACKAMAS-032': 'CLACKAMAS-032 032', 'CLACKAMAS-033': 'CLACKAMAS-033 033', 'CLACKAMAS-040': 'CLACKAMAS-040 040', 'CLACKAMAS-051': 'CLACKAMAS-051 051', 'CLACKAMAS-052': 'CLACKAMAS-052 052', 'CLACKAMAS-053': 'CLACKAMAS-053 053', 'CLACKAMAS-055': 'CLACKAMAS-055 055', 'CLACKAMAS-070 & 071': 'CLACKAMAS-070/071 070/071', 'CLACKAMAS-072': 'CLACKAMAS-072 072', 'CLACKAMAS-073': 'CLACKAMAS-073 073', 'CLACKAMAS-074': 'CLACKAMAS-074 074', 'CLACKAMAS-099 & 100': 'CLACKAMAS-099/100 099/100', 'CLACKAMAS-101': 'CLACKAMAS-101 101', 'CLACKAMAS-102': 'CLACKAMAS-102 102', 'CLACKAMAS-103 & 104': 'CLACKAMAS-103/104 103/104', 'CLACKAMAS-105': 'CLACKAMAS-105 105', 'CLACKAMAS-106': 'CLACKAMAS-106 106', 'CLACKAMAS-107': 'CLACKAMAS-107 107', 'CLACKAMAS-108': 'CLACKAMAS-108 108', 'CLACKAMAS-111': 'CLACKAMAS-111 111', 'CLACKAMAS-122': 'CLACKAMAS-122 122', 'CLACKAMAS-124': 'CLACKAMAS-124 124', 'CLACKAMAS-125': 'CLACKAMAS-125 125', 'CLACKAMAS-127': 'CLACKAMAS-127 127', 'CLACKAMAS-131': 'CLACKAMAS-131 131', 'CLACKAMAS-132': 'CLACKAMAS-132 132', 'CLACKAMAS-133': 'CLACKAMAS-133 133', 'CLACKAMAS-134': 'CLACKAMAS-134 134', 'CLACKAMAS-135': 'CLACKAMAS-135 135', 'CLACKAMAS-151': 'CLACKAMAS-151 151', 'CLACKAMAS-153': 'CLACKAMAS-153 153', 'CLACKAMAS-154': 'CLACKAMAS-154 154', 'CLACKAMAS-155': 'CLACKAMAS-155 155', 'CLACKAMAS-156': 'CLACKAMAS-156 156', 'CLACKAMAS-157': 'CLACKAMAS-157 157', 'CLACKAMAS-159': 'CLACKAMAS-159 159', 'CLACKAMAS-160': 'CLACKAMAS-160 160', 'CLACKAMAS-201': 'CLACKAMAS-201 201', 'CLACKAMAS-202': 'CLACKAMAS-202 202', 'CLACKAMAS-203': 'CLACKAMAS-203 203', 'CLACKAMAS-204': 'CLACKAMAS-204 204', 'CLACKAMAS-251 & 252': 'CLACKAMAS-251/252 251/252', 'CLACKAMAS-280': 'CLACKAMAS-280 280', 'CLACKAMAS-281': 'CLACKAMAS-281 281', 'CLACKAMAS-282': 'CLACKAMAS-282 282', 'CLACKAMAS-283': 'CLACKAMAS-283 283', 'CLACKAMAS-284': 'CLACKAMAS-284 284', 'CLACKAMAS-320': 'CLACKAMAS-320 320', 'CLACKAMAS-321': 'CLACKAMAS-321 321', 'CLACKAMAS-322': 'CLACKAMAS-322 322', 'CLACKAMAS-323': 'CLACKAMAS-323 323', 'CLACKAMAS-330': 'CLACKAMAS-330 330', 'CLACKAMAS-331': 'CLACKAMAS-331 331', 'CLACKAMAS-332': 'CLACKAMAS-332 332', 'CLACKAMAS-333': 'CLACKAMAS-333 333', 'CLACKAMAS-350': 'CLACKAMAS-350 350', 'CLACKAMAS-351': 'CLACKAMAS-351 351', 'CLACKAMAS-352': 'CLACKAMAS-352 352', 'CLACKAMAS-359': 'CLACKAMAS-359 359', 'CLACKAMAS-360': 'CLACKAMAS-360 360', 'CLACKAMAS-361 & 362': 'CLACKAMAS-361/362 361/362', 'CLACKAMAS-363': 'CLACKAMAS-363 363', 'CLACKAMAS-364': 'CLACKAMAS-364 364', 'CLACKAMAS-370': 'CLACKAMAS-370 370', 'CLACKAMAS-371': 'CLACKAMAS-371 371', 'CLACKAMAS-372': 'CLACKAMAS-372 372', 'CLACKAMAS-373': 'CLACKAMAS-373 373', 'CLACKAMAS-400': 'CLACKAMAS-400 400', 'CLACKAMAS-401': 'CLACKAMAS-401 401', 'CLACKAMAS-402': 'CLACKAMAS-402 402', 'CLACKAMAS-403': 'CLACKAMAS-403 403', 'CLACKAMAS-404': 'CLACKAMAS-404 404', 'CLACKAMAS-405': 'CLACKAMAS-405 405', 'CLACKAMAS-406': 'CLACKAMAS-406 406', 'CLACKAMAS-410': 'CLACKAMAS-410 410', 'CLACKAMAS-411': 'CLACKAMAS-411 411', 'CLACKAMAS-412': 'CLACKAMAS-412 412', 'CLACKAMAS-413': 'CLACKAMAS-413 413', 'CLACKAMAS-414': 'CLACKAMAS-414 414', 'CLACKAMAS-415': 'CLACKAMAS-415 415', 'CLACKAMAS-416': 'CLACKAMAS-416 416', 'CLACKAMAS-417 & 418': 'CLACKAMAS-417/418 417/418', 'CLACKAMAS-419': 'CLACKAMAS-419 419', 'CLACKAMAS-420': 'CLACKAMAS-420 420', 'CLACKAMAS-421': 'CLACKAMAS-421 421', 'CLACKAMAS-422': 'CLACKAMAS-422 422', 'CLACKAMAS-500': 'CLACKAMAS-500 500', 'CLACKAMAS-501': 'CLACKAMAS-501 501', 'CLACKAMAS-502': 'CLACKAMAS-502 502', 'CLACKAMAS-503': 'CLACKAMAS-503 503', 'CLACKAMAS-504': 'CLACKAMAS-504 504', 'CLACKAMAS-505': 'CLACKAMAS-505 505', 'CLACKAMAS-506': 'CLACKAMAS-506 506', 'CLACKAMAS-510': 'CLACKAMAS-510 510', 'CLACKAMAS-511': 'CLACKAMAS-511 511', 'CLACKAMAS-512': 'CLACKAMAS-512 512', 'CLACKAMAS-513': 'CLACKAMAS-513 513', 'CLACKAMAS-514': 'CLACKAMAS-514 514', 'CLACKAMAS-515': 'CLACKAMAS-515 515', 'CLACKAMAS-516': 'CLACKAMAS-516 516', 'CLACKAMAS-517': 'CLACKAMAS-517 517', 'CLACKAMAS-518': 'CLACKAMAS-518 518', 'CLACKAMAS-519': 'CLACKAMAS-519 519', 'CLATSOP-01 - ASTORIA': 'CLATSOP-ASTORIA 01', 'CLATSOP-02 - ASTORIA': 'CLATSOP-ASTORIA 02', 'CLATSOP-03 - ASTORIA': 'CLATSOP-ASTORIA 03', 'CLATSOP-04 - ASTORIA': 'CLATSOP-ASTORIA 04', 'CLATSOP-05 - ASTORIA': 'CLATSOP-ASTORIA 05', 'CLATSOP-06 - ASTORIA': 'CLATSOP-ASTORIA 06', 'CLATSOP-07 - ASTORIA': 'CLATSOP-ASTORIA 07', 'CLATSOP-08 - ASTORIA': 'CLATSOP-ASTORIA 08', 'CLATSOP-20 - CANNON BEACH': 'CLATSOP-CANNON BEACH 20', 'CLATSOP-21 - CHADWELL': 'CLATSOP-CHADWELL 21', 'CLATSOP-22 - NORTH CLATSOP': 'CLATSOP-NORTH CLATSOP 22', 'CLATSOP-23 - SOUTH CLATSOP': 'CLATSOP-SOUTH CLATSOP 23', 'CLATSOP-24 - ELSIE': 'CLATSOP-ELSIE 24', 'CLATSOP-25 - GEARHART': 'CLATSOP-GEARHART 25', 'CLATSOP-26 - HAMLET': 'CLATSOP-HAMLET 26', 'CLATSOP-28 - JEWELL': 'CLATSOP-JEWELL 28', 'CLATSOP-29 - JOHN DAY': 'CLATSOP-JOHN DAY 29', 'CLATSOP-30 - KNAPPA': 'CLATSOP-KNAPPA 30', 'CLATSOP-31 - LEWIS AND CLARK': 'CLATSOP-LEWIS & CLARK 31', 'CLATSOP-35 - NECANICUM': 'CLATSOP-NECANICUM 35', 'CLATSOP-36 - OLNEY': 'CLATSOP-OLNEY 36', 'CLATSOP-37 - SEASIDE': 'CLATSOP-SEASIDE 37', 'CLATSOP-38 - SEASIDE': 'CLATSOP-SEASIDE 38', 'CLATSOP-39 - SEASIDE': 'CLATSOP-SEASIDE 39', 'CLATSOP-40 - SEASIDE': 'CLATSOP-SEASIDE 40', 'CLATSOP-43 - STANLEY ACRES': 'CLATSOP-STANLEY ACRES 43', 'CLATSOP-44 - SVENSEN': 'CLATSOP-SVENSEN 44', 'CLATSOP-45 - ARCH CAPE': 'CLATSOP-ARCH CAPE 45', 'CLATSOP-46 - WALLUSKI': 'CLATSOP-WALLUSKI 46', 'CLATSOP-47 - WARRENTON': 'CLATSOP-WARRENTON 47', 'CLATSOP-48 - WARRENTON': 'CLATSOP-WARRENTON 48', 'CLATSOP-50 - WESTPORT': 'CLATSOP-WESTPORT 50', 'CLATSOP-52 - HILLCREST': 'CLATSOP-HILLCREST 52', 'COLUMBIA-01 CITY OF CLATSKANIE': 'COLUMBIA-CITY OF CLATSKANIE 01', 'COLUMBIA-04 N CLATSKANIE-MARSHLAND': 'COLUMBIA-N CLATSKANIE-MARSHLAND 04', 'COLUMBIA-05 S CLATSKANIE': 'COLUMBIA-S CLATSKANIE 05', 'COLUMBIA-06 DELENA': 'COLUMBIA-DELENA 06', 'COLUMBIA-08 GOBLE': 'COLUMBIA-GOBLE 08', 'COLUMBIA-10 MILTON': 'COLUMBIA-MILTON 10', 'COLUMBIA-12 MCNULTY': 'COLUMBIA-MCNULTY 12', 'COLUMBIA-14 QUINCY': 'COLUMBIA-QUINCY 14', 'COLUMBIA-15 CITY OF RAINIER': 'COLUMBIA-RAINIER 15', 'COLUMBIA-17 W RAINIER': 'COLUMBIA-W RAINIER 17', 'COLUMBIA-18 E RAINIER': 'COLUMBIA-E RAINIER 18', 'COLUMBIA-19 S WARREN': 'COLUMBIA-S WARREN 19', 'COLUMBIA-20 N WARREN': 'COLUMBIA-N WARREN 20', 'COLUMBIA-21 CITY ST HELENS ONE': 'COLUMBIA-ST HELENS ONE 21', 'COLUMBIA-24 CITY ST HELENS FOUR': 'COLUMBIA-ST HELENS FOUR 24', 'COLUMBIA-26 CITY ST HELENS SIX': 'COLUMBIA-ST HELENS SIX 26', 'COLUMBIA-31 CITY OF SCAPPOOSE ONE': 'COLUMBIA-SCAPPOOSE ONE 31', 'COLUMBIA-32 CITY OF SCAPPOOSE TWO': 'COLUMBIA-SCAPPOOSE TWO 32', 'COLUMBIA-34 CITY OF SCAPPOOSE FOUR': 'COLUMBIA-SCAPPOOSE FOUR 34', 'COLUMBIA-35 SE SCAPPOOSE': 'COLUMBIA-SE SCAPPOOSE 35', 'COLUMBIA-36 CANYON': 'COLUMBIA-CANYON 36', 'COLUMBIA-38 W SCAPPOOSE': 'COLUMBIA-W SCAPPOOSE 38', 'COLUMBIA-39 SAUVIE ISLAND': 'COLUMBIA-SAUVIE ISLAND 39', 'COLUMBIA-40 YANKTON': 'COLUMBIA-YANKTON 40', 'COLUMBIA-41 CITY OF VERNONIA': 'COLUMBIA-CITY OF VERNONIA 41', 'COLUMBIA-44 RURAL VERNONIA': 'COLUMBIA-RURAL VERNONIA 44', 'COLUMBIA-46 APIARY': 'COLUMBIA-APIARY 46', 'COLUMBIA-47 CHAPMAN': 'COLUMBIA-CHAPMAN 47', 'COLUMBIA-48 S DEER ISLAND': 'COLUMBIA-S DEER ISLAND 48', 'COLUMBIA-49 N DEER ISLAND': 'COLUMBIA-N DEER ISLAND 49', 'COLUMBIA-50 CITY OF PRESCOTT': 'COLUMBIA-PRESCOTT 50', 'COLUMBIA-51 MIST': 'COLUMBIA-MIST 51', 'COLUMBIA-53 CITY OF COLUMBIA CITY': 'COLUMBIA-CITY OF COLUMBIA CITY 53', 'COOS-PRECINCT 01': 'COOS-LAKESIDE CITY/RURAL 01', 'COOS-PRECINCT 02': 'COOS-NORTH BEND RURAL 02', 'COOS-PRECINCT 03': 'COOS-NORTH BEND CITY WEST 03', 'COOS-PRECINCT 04': 'COOS-NORTH BEND CITY EAST 04', 'COOS-PRECINCT 05': 'COOS-COOS BAY CITY WEST 05', 'COOS-PRECINCT 06': 'COOS-COOS BAY CITY MID 06', 'COOS-PRECINCT 07': 'COOS-COOS BAY CITY EAST 07', 'COOS-PRECINCT 08': 'COOS-COOS BAY RURAL EAST 08', 'COOS-PRECINCT 09': 'COOS-COOS BAY RURAL WEST 09', 'COOS-PRECINCT 10': 'COOS-COOS BAY RURAL 10', 'COOS-PRECINCT 11': 'COOS-COQUILLE RURAL 11', 'COOS-PRECINCT 12': 'COOS-COQUILLE CITY 12', 'COOS-PRECINCT 13': 'COOS-MYRTLE POINT CITY 13', 'COOS-PRECINCT 14': 'COOS-MYRTLE POINT RURAL 14', 'COOS-PRECINCT 15': 'COOS-POWERS CITY/RURAL 15', 'COOS-PRECINCT 16': 'COOS-BANDON CITY 16', 'COOS-PRECINCT 17': 'COOS-BANDON RURAL 17 17', 'COOS-PRECINCT 18': 'COOS-BANDON RURAL 18 18', 'CROOK-PRECINCT 01': 'CROOK-PRECINCT 01 01', 'CROOK-PRECINCT 02': 'CROOK-PRECINCT 02 02', 'CROOK-PRECINCT 03': 'CROOK-PRECINCT 03 03', 'CROOK-PRECINCT 04': 'CROOK-PRECINCT 04 04', 'CROOK-PRECINCT 05': 'CROOK-PRECINCT 05 05', 'CROOK-PRECINCT 06': 'CROOK-PRECINCT 06 06', 'CROOK-PRECINCT 07': 'CROOK-PRECINCT 07 07', 'CROOK-PRECINCT 08': 'CROOK-PRECINCT 08 08', 'CROOK-PRECINCT 09': 'CROOK-PRECINCT 09 09', 'CROOK-PRECINCT 10': 'CROOK-PRECINCT 10 10', 'CROOK-PRECINCT 11': 'CROOK-PRECINCT 11 11', 'CROOK-PRECINCT 12': 'CROOK-PRECINCT 12 12', 'CROOK-PRECINCT 13': 'CROOK-PRECINCT 13 13', 'CROOK-PRECINCT 14': 'CROOK-PRECINCT 14 14', 'CROOK-PRECINCT 15': 'CROOK-PRECINCT 15 15', 'CROOK-PRECINCT 16': 'CROOK-PRECINCT 16 16', 'CROOK-PRECINCT 17': 'CROOK-PRECINCT 17 17', 'CURRY-PRECINCT 01': 'CURRY-AGNESS 01', 'CURRY-PRECINCT 02': 'CURRY-SOUTH CHETCO 02', 'CURRY-PRECINCT 03': 'CURRY-FLORAS CREEK 03', 'CURRY-PRECINCT 04': 'CURRY-SIXES 04', 'CURRY-PRECINCT 05': 'CURRY-ELK RIVER 05', 'CURRY-PRECINCT 06': 'CURRY-PORT ORFORD 1 06', 'CURRY-PRECINCT 07': 'CURRY-PORT ORFORD 2 07', 'CURRY-PRECINCT 08': 'CURRY-OPHIR 08', 'CURRY-PRECINCT 09': 'CURRY-WEDDERBURN 09', 'CURRY-PRECINCT 10': 'CURRY-N GOLD BEACH 10', 'CURRY-PRECINCT 11': 'CURRY-S GOLD BEACH 11', 'CURRY-PRECINCT 12': 'CURRY-HUNTER CREEK 12', 'CURRY-PRECINCT 13': 'CURRY-PISTOL RIVER 13', 'CURRY-PRECINCT 14': 'CURRY-HARRIS CREEK 14', 'CURRY-PRECINCT 15': 'CURRY-BROOKINGS 1 15', 'CURRY-PRECINCT 16': 'CURRY-BROOKINGS 2 16', 'CURRY-PRECINCT 17': 'CURRY-BROOKINGS 3 17', 'CURRY-PRECINCT 18': 'CURRY-NORTH CHETCO 18', 'CURRY-PRECINCT 19': 'CURRY-WINCHUCK 19', 'CURRY-PRECINCT 20': 'CURRY-JERRYS FLAT 20', 'CURRY-PRECINCT 21': 'CURRY-BROOKINGS 4 21', 'CURRY-PRECINCT 22': 'CURRY-WEST HARBOR 22', 'CURRY-PRECINCT 23': 'CURRY-LOWER HARBOR 23', 'CURRY-PRECINCT 24': 'CURRY-SW HARBOR 24', 'CURRY-PRECINCT 25': 'CURRY-EAST HARBOR 25', 'CURRY-PRECINCT 26': 'CURRY-CAPE FERRELO 26', 'CURRY-PRECINCT 27': 'CURRY-UPPER CHETCO 27', 'DESCHUTES-PRECINCT 01': 'DESCHUTES-DESCHUTES 01 01', 'DESCHUTES-PRECINCT 02': 'DESCHUTES-DESCHUTES 02 02', 'DESCHUTES-PRECINCT 03': 'DESCHUTES-DESCHUTES 03 03', 'DESCHUTES-PRECINCT 04': 'DESCHUTES-DESCHUTES 04 04', 'DESCHUTES-PRECINCT 05': 'DESCHUTES-DESCHUTES 05 05', 'DESCHUTES-PRECINCT 06': 'DESCHUTES-DESCHUTES 06 06', 'DESCHUTES-PRECINCT 07': 'DESCHUTES-DESCHUTES 07 07', 'DESCHUTES-PRECINCT 08': 'DESCHUTES-DESCHUTES 08 08', 'DESCHUTES-PRECINCT 09': 'DESCHUTES-DESCHUTES 09 09', 'DESCHUTES-PRECINCT 10': 'DESCHUTES-DESCHUTES 10 10', 'DESCHUTES-PRECINCT 11': 'DESCHUTES-DESCHUTES 11 11', 'DESCHUTES-PRECINCT 12': 'DESCHUTES-DESCHUTES 12 12', 'DESCHUTES-PRECINCT 13': 'DESCHUTES-DESCHUTES 13 13', 'DESCHUTES-PRECINCT 14': 'DESCHUTES-DESCHUTES 14 14', 'DESCHUTES-PRECINCT 15': 'DESCHUTES-DESCHUTES 15 15', 'DESCHUTES-PRECINCT 16': 'DESCHUTES-DESCHUTES 16 16', 'DESCHUTES-PRECINCT 17': 'DESCHUTES-DESCHUTES 17 17', 'DESCHUTES-PRECINCT 18': 'DESCHUTES-DESCHUTES 18 18', 'DESCHUTES-PRECINCT 19': 'DESCHUTES-DESCHUTES 19 19', 'DESCHUTES-PRECINCT 20': 'DESCHUTES-DESCHUTES 20 20', 'DESCHUTES-PRECINCT 21': 'DESCHUTES-DESCHUTES 21 21', 'DESCHUTES-PRECINCT 22': 'DESCHUTES-DESCHUTES 22 22', 'DESCHUTES-PRECINCT 23': 'DESCHUTES-DESCHUTES 23 23', 'DESCHUTES-PRECINCT 24': 'DESCHUTES-DESCHUTES 24 24', 'DESCHUTES-PRECINCT 25': 'DESCHUTES-DESCHUTES 25 25', 'DESCHUTES-PRECINCT 26': 'DESCHUTES-DESCHUTES 26 26', 'DESCHUTES-PRECINCT 27': 'DESCHUTES-DESCHUTES 27 27', 'DESCHUTES-PRECINCT 28': 'DESCHUTES-DESCHUTES 28 28', 'DESCHUTES-PRECINCT 29': 'DESCHUTES-DESCHUTES 29 29', 'DESCHUTES-PRECINCT 30': 'DESCHUTES-DESCHUTES 30 30', 'DESCHUTES-PRECINCT 31': 'DESCHUTES-DESCHUTES 31 31', 'DESCHUTES-PRECINCT 32': 'DESCHUTES-DESCHUTES 32 32', 'DESCHUTES-PRECINCT 33': 'DESCHUTES-DESCHUTES 33 33', 'DESCHUTES-PRECINCT 34': 'DESCHUTES-DESCHUTES 34 34', 'DESCHUTES-PRECINCT 35': 'DESCHUTES-DESCHUTES 35 35', 'DESCHUTES-PRECINCT 36': 'DESCHUTES-DESCHUTES 36 36', 'DESCHUTES-PRECINCT 37': 'DESCHUTES-DESCHUTES 37 37', 'DESCHUTES-PRECINCT 38': 'DESCHUTES-DESCHUTES 38 38', 'DESCHUTES-PRECINCT 39': 'DESCHUTES-DESCHUTES 39 39', 'DESCHUTES-PRECINCT 40': 'DESCHUTES-DESCHUTES 40 40', 'DESCHUTES-PRECINCT 41': 'DESCHUTES-DESCHUTES 41 41', 'DESCHUTES-PRECINCT 42': 'DESCHUTES-DESCHUTES 42 42', 'DESCHUTES-PRECINCT 43': 'DESCHUTES-DESCHUTES 43 43', 'DESCHUTES-PRECINCT 44': 'DESCHUTES-DESCHUTES 44 44', 'DESCHUTES-PRECINCT 45': 'DESCHUTES-DESCHUTES 45 45', 'DESCHUTES-PRECINCT 46': 'DESCHUTES-DESCHUTES 46 46', 'DESCHUTES-PRECINCT 47': 'DESCHUTES-DESCHUTES 47 47', 'DESCHUTES-PRECINCT 48': 'DESCHUTES-DESCHUTES 48 48', 'DESCHUTES-PRECINCT 49': 'DESCHUTES-DESCHUTES 49 49', 'DESCHUTES-PRECINCT 50': 'DESCHUTES-DESCHUTES 50 50', 'DOUGLAS-PRECINCT 01': 'DOUGLAS-UNINCORP REEDSPORT 01', 'DOUGLAS-PRECINCT 02': 'DOUGLAS-ROSEBURG CITY W1 02', 'DOUGLAS-PRECINCT 03': 'DOUGLAS-REEDSPORT CITY 03', 'DOUGLAS-PRECINCT 04': 'DOUGLAS-NORTH COUNTY 04', 'DOUGLAS-PRECINCT 05': 'DOUGLAS-EAST COUNTY 05', 'DOUGLAS-PRECINCT 06': 'DOUGLAS-UNINC N ROSEBURG 06', 'DOUGLAS-PRECINCT 07': 'DOUGLAS-ROSEBURG CITY W2 07', 'DOUGLAS-PRECINCT 08': 'DOUGLAS-ROSEBURG UNINCORP 08', 'DOUGLAS-PRECINCT 09': 'DOUGLAS-ROSEBURG CITY W3 09', 'DOUGLAS-PRECINCT 10': 'DOUGLAS-ELKTON CITY 10', 'DOUGLAS-PRECINCT 11': 'DOUGLAS-DRAIN CITY 11', 'DOUGLAS-PRECINCT 12': 'DOUGLAS-NW OF ROSEBURG 12', 'DOUGLAS-PRECINCT 13': 'DOUGLAS-COUNTY WEST 13', 'DOUGLAS-PRECINCT 14': 'DOUGLAS-YONCALLA CITY 14', 'DOUGLAS-PRECINCT 15': 'DOUGLAS-OAKLAND CITY 15', 'DOUGLAS-PRECINCT 16': 'DOUGLAS-COUNTY SW 16', 'DOUGLAS-PRECINCT 17': 'DOUGLAS-SUTHERLIN CITY 17', 'DOUGLAS-PRECINCT 18': 'DOUGLAS-ROSEBURG CITY W1 18', 'DOUGLAS-PRECINCT 19': 'DOUGLAS-NEAR MYRTLE CREEK 19', 'DOUGLAS-PRECINCT 20': 'DOUGLAS-SE OF ROSEBURG 20', 'DOUGLAS-PRECINCT 21': 'DOUGLAS-COUNTY SOUTH 21', 'DOUGLAS-PRECINCT 22': 'DOUGLAS-ROSEBURG CITY W4 22', 'DOUGLAS-PRECINCT 23': 'DOUGLAS-WINSTON CITY 23', 'DOUGLAS-PRECINCT 24': 'DOUGLAS-WINSTON CITY 24', 'DOUGLAS-PRECINCT 25': 'DOUGLAS-CANYONVILLE CITY 25', 'DOUGLAS-PRECINCT 26': 'DOUGLAS-MYRTLE CREEK 26', 'DOUGLAS-PRECINCT 27': 'DOUGLAS-GLENDALE CITY 27', 'DOUGLAS-PRECINCT 28': 'DOUGLAS-RIDDLE CITY 28', 'GILLIAM-PRECINCT 1': 'GILLIAM-ARLINGTON 01', 'GILLIAM-PRECINCT 6': 'GILLIAM-EAST CONDON 06', 'GILLIAM-PRECINCT 7': 'GILLIAM-WEST CONDON 07', 'GRANT-PRECINCT 3 - JOHN DAY VALLEY': 'GRANT-JOHN DAY 03', 'GRANT-PRECINCT 16 - SOUTH FORK': 'GRANT-SOUTH FORK 16', 'GRANT-PRECINCT 17 - LONG CREEK VALLEY': 'GRANT-LONG CREEK 17', 'GRANT-PRECINCT 4 - UNION': 'GRANT-UNION 04', 'GRANT-PRECINCT 8 - NORTH FORK': 'GRANT-NORTH FORK 08', 'HARNEY-PRECINCT 21': 'HARNEY-BURNS 21', 'HARNEY-PRECINCT 25': 'HARNEY-HINES 25', 'HARNEY-PRECINCT 26': 'HARNEY-RURAL HARNEY 26', 'HOOD RIVER-1': 'HOOD RIVER-PRECINCT 01 01', 'HOOD RIVER-2': 'HOOD RIVER-PRECINCT 02 02', 'HOOD RIVER-3': 'HOOD RIVER-PRECINCT 03 03', 'HOOD RIVER-4': 'HOOD RIVER-PRECINCT 04 04', 'HOOD RIVER-5': 'HOOD RIVER-PRECINCT 05 05', 'HOOD RIVER-6': 'HOOD RIVER-PRECINCT 06 06', 'HOOD RIVER-7': 'HOOD RIVER-PRECINCT 07 07', 'HOOD RIVER-8': 'HOOD RIVER-PRECINCT 08 08', 'HOOD RIVER-9': 'HOOD RIVER-PRECINCT 09 09', 'HOOD RIVER-10': 'HOOD RIVER-PRECINCT 10 10', 'HOOD RIVER-11': 'HOOD RIVER-PRECINCT 11 11', 'HOOD RIVER-12': 'HOOD RIVER-PRECINCT 12 12', 'HOOD RIVER-13': 'HOOD RIVER-PRECINCT 13 13', 'JACKSON-PRECINCT 001': 'JACKSON-ASHLAND 01', 'JACKSON-PRECINCT 002': 'JACKSON-ASHLAND 02', 'JACKSON-PRECINCT 003': 'JACKSON-BUTTE FALLS 03', 'JACKSON-PRECINCT 004': 'JACKSON-CENTRAL POINT 04', 'JACKSON-PRECINCT 005': 'JACKSON-CENTRAL POINT 05', 'JACKSON-PRECINCT 006': 'JACKSON-CENTRAL POINT 06', 'JACKSON-PRECINCT 007': 'JACKSON-CENTRAL POINT 07', 'JACKSON-PRECINCT 008': 'JACKSON-EAGLE POINT 08', 'JACKSON-PRECINCT 009': 'JACKSON-GOLD HILL 09', 'JACKSON-PRECINCT 010': 'JACKSON-JACKSONVILLE 10', 'JACKSON-PRECINCT 011': 'JACKSON-MEDFORD 11', 'JACKSON-PRECINCT 012': 'JACKSON-MEDFORD 12', 'JACKSON-PRECINCT 013': 'JACKSON-MEDFORD 13', 'JACKSON-PRECINCT 014': 'JACKSON-MEDFORD 14', 'JACKSON-PRECINCT 015': 'JACKSON-MEDFORD 15', 'JACKSON-PRECINCT 016': 'JACKSON-MEDFORD 16', 'JACKSON-PRECINCT 017': 'JACKSON-MEDFORD 17', 'JACKSON-PRECINCT 018': 'JACKSON-MEDFORD 18', 'JACKSON-PRECINCT 019': 'JACKSON-MEDFORD 19', 'JACKSON-PRECINCT 020': 'JACKSON-MEDFORD 20', 'JACKSON-PRECINCT 021': 'JACKSON-MEDFORD 21', 'JACKSON-PRECINCT 022': 'JACKSON-PHOENIX 22', 'JACKSON-PRECINCT 023': 'JACKSON-ROGUE RIVER 23', 'JACKSON-PRECINCT 024': 'JACKSON-SHADY COVE 24', 'JACKSON-PRECINCT 025': 'JACKSON-TALENT 25', 'JACKSON-PRECINCT 026': 'JACKSON-ASHLAND PINEHURST 26', 'JACKSON-PRECINCT 027': 'JACKSON-PROSPECT 27', 'JACKSON-PRECINCT 028': 'JACKSON-SHADY COVE 28', 'JACKSON-PRECINCT 029': 'JACKSON-MEDFORD W RURAL 29', 'JACKSON-PRECINCT 030': 'JACKSON-LAKE CREEK 30', 'JACKSON-PRECINCT 031': 'JACKSON-WHITE CITY RURAL 31', 'JACKSON-PRECINCT 032': 'JACKSON-JACKSONVILLE E RURAL 32', 'JACKSON-PRECINCT 033': 'JACKSON-JACKSONVILLE W RURAL 33', 'JACKSON-PRECINCT 034': 'JACKSON-EAGLE POINT RURAL 34', 'JACKSON-PRECINCT 035': 'JACKSON-RUCH 35', 'JACKSON-PRECINCT 036': 'JACKSON-WHITE CITY 36', 'JACKSON-PRECINCT 037': 'JACKSON-TALENT RURAL 37', 'JACKSON-PRECINCT 038': 'JACKSON-MEDFORD E RURAL 38', 'JACKSON-PRECINCT 039': 'JACKSON-PHOENIX RURAL 39', 'JACKSON-PRECINCT 040': 'JACKSON-CENTRAL POINT RURAL 40', 'JACKSON-PRECINCT 041': 'JACKSON-GOLD HILL RURAL 41', 'JACKSON-PRECINCT 042': 'JACKSON-SAMS VALLEY 42', 'JACKSON-PRECINCT 043': 'JACKSON-WIMER 43', 'JACKSON-PRECINCT 044': 'JACKSON-ROGUE RIVER RURAL 44', 'JACKSON-PRECINCT 045': 'JACKSON-APPLEGATE 45', 'JEFFERSON-NO. 1 HAYSTACK': 'JEFFERSON-HAYSTACK 01', 'JEFFERSON-NO. 2 CROOKED RIVER RANCH': 'JEFFERSON-CROOKED RIVER RANCH 02', 'JEFFERSON-NO. 5 ASHWOOD': 'JEFFERSON-ASHWOOD 05', 'JEFFERSON-NO. 6 KUTCHER': 'JEFFERSON-KUTCHER 06', 'JEFFERSON-NO. 8 LYLE GAP': 'JEFFERSON-LYLE GAP 08', 'JEFFERSON-NO. 11 METOLIUS': 'JEFFERSON-METOLIUS 11', 'JEFFERSON-NO. 13 EAST MADRAS': 'JEFFERSON-EAST MADRAS 13', 'JEFFERSON-NO. 14 WARM SPRINGS': 'JEFFERSON-WARM SPRINGS 14', 'JEFFERSON-NO. 16 CAMP SHERMAN': 'JEFFERSON-CAMP SHERMAN 16', 'JEFFERSON-NO. 17 CULVER': 'JEFFERSON-CULVER 17', 'JEFFERSON-NO. 18 ROUND BUTTE': 'JEFFERSON-ROUND BUTTE 18', 'JEFFERSON-NO. 19 WEST MADRAS': 'JEFFERSON-WEST MADRAS 19', 'JEFFERSON-NO. 20 CIRCLE M': 'JEFFERSON-CIRCLE M 20', 'JEFFERSON-NO. 21 CENTRAL': 'JEFFERSON-CENTRAL 21', 'JEFFERSON-NO. 22 SUNSET': 'JEFFERSON-SUNSET 22', 'JOSEPHINE-PRECINCT 01': 'JOSEPHINE-HIGHLAND 01', 'JOSEPHINE-PRECINCT 02': 'JOSEPHINE-GREENWOOD SOUTH 02', 'JOSEPHINE-PRECINCT 03': 'JOSEPHINE-LINCOLN 03', 'JOSEPHINE-PRECINCT 04': 'JOSEPHINE-CENTRAL 04', 'JOSEPHINE-PRECINCT 05': 'JOSEPHINE-RIVERSIDE 05', 'JOSEPHINE-PRECINCT 06': 'JOSEPHINE-WEBSTER 06', 'JOSEPHINE-PRECINCT 07': 'JOSEPHINE-GREENWOOD 07', 'JOSEPHINE-PRECINCT 08': 'JOSEPHINE-RIVERSIDE SOUTH 08', 'JOSEPHINE-PRECINCT 09': 'JOSEPHINE-SCOVILLE 09', 'JOSEPHINE-PRECINCT 10': 'JOSEPHINE-ROBERTSON 10', 'JOSEPHINE-PRECINCT 11': 'JOSEPHINE-BEACON 11', 'JOSEPHINE-PRECINCT 12': 'JOSEPHINE-RIVER ROAD 12', 'JOSEPHINE-PRECINCT 13': 'JOSEPHINE-HAMILTON 13', 'JOSEPHINE-PRECINCT 14': 'JOSEPHINE-FORT VANNOY 14', 'JOSEPHINE-PRECINCT 15': 'JOSEPHINE-REDWOOD 15', 'JOSEPHINE-PRECINCT 16': 'JOSEPHINE-CAMP JOY 16', 'JOSEPHINE-PRECINCT 17': 'JOSEPHINE-DIMMICK 17', 'JOSEPHINE-PRECINCT 18': 'JOSEPHINE-COLLEGE 18', 'JOSEPHINE-PRECINCT 19': 'JOSEPHINE-JEROME PRAIRIE 19', 'JOSEPHINE-PRECINCT 20': 'JOSEPHINE-GRAVE CREEK 20', 'JOSEPHINE-PRECINCT 21': 'JOSEPHINE-CLOVERLAWN 21', 'JOSEPHINE-PRECINCT 22': 'JOSEPHINE-NEW HOPE 22', 'JOSEPHINE-PRECINCT 23': 'JOSEPHINE-LOUSE CREEK 23', 'JOSEPHINE-PRECINCT 24': 'JOSEPHINE-ALLENDALE 24', 'JOSEPHINE-PRECINCT 25': 'JOSEPHINE-JONES CREEK 25', 'JOSEPHINE-PRECINCT 26': 'JOSEPHINE-BOARD SHANTY 26', 'JOSEPHINE-PRECINCT 27': 'JOSEPHINE-WOLF CREEK 27', 'JOSEPHINE-PRECINCT 28': 'JOSEPHINE-KERBY 28', 'JOSEPHINE-PRECINCT 29': 'JOSEPHINE-WEST COLONIAL 29', 'JOSEPHINE-PRECINCT 30': 'JOSEPHINE-CAVE JUNCTION 30', 'JOSEPHINE-PRECINCT 31': 'JOSEPHINE-BRIDGEVIEW 31', 'JOSEPHINE-PRECINCT 32': 'JOSEPHINE-MT. PEAVINE 32', 'JOSEPHINE-PRECINCT 33': 'JOSEPHINE-FERRY 33', 'JOSEPHINE-PRECINCT 34': 'JOSEPHINE-ILLINOIS 34', 'JOSEPHINE-PRECINCT 35': 'JOSEPHINE-MERLIN 35', 'JOSEPHINE-PRECINCT 36': 'JOSEPHINE-FINDLEY BEND 36', 'JOSEPHINE-PRECINCT 37': 'JOSEPHINE-SELMA 37', 'JOSEPHINE-PRECINCT 38': 'JOSEPHINE-SLATE CREEK 38', 'JOSEPHINE-PRECINCT 39': 'JOSEPHINE-LUCKY QUEEN 39', 'JOSEPHINE-PRECINCT 40': 'JOSEPHINE-WILLIAMS 40', 'JOSEPHINE-PRECINCT 41': 'JOSEPHINE-COLONIAL 41', 'JOSEPHINE-PRECINCT 42': 'JOSEPHINE-NORTH VALLEY 42', 'JOSEPHINE-PRECINCT 43': 'JOSEPHINE-PLEASANT VALLEY 43', 'JOSEPHINE-PRECINCT 44': 'JOSEPHINE-ILLINOIS RIVER 44', 'JOSEPHINE-PRECINCT 45': 'JOSEPHINE-NORTH APPLEGATE 45', 'JOSEPHINE-PRECINCT 46': 'JOSEPHINE-POWELL CREEK 46', 'JOSEPHINE-PRECINCT 47': 'JOSEPHINE-CRESTVIEW 47', 'KLAMATH-PRECINCT 01': 'KLAMATH-PRECINCT 01 01', 'KLAMATH-PRECINCT 02': 'KLAMATH-PRECINCT 02 02', 'KLAMATH-PRECINCT 03': 'KLAMATH-PRECINCT 03 03', 'KLAMATH-PRECINCT 04': 'KLAMATH-PRECINCT 04 04', 'KLAMATH-PRECINCT 05': 'KLAMATH-PRECINCT 05 05', 'KLAMATH-PRECINCT 06': 'KLAMATH-PRECINCT 06 06', 'KLAMATH-PRECINCT 07': 'KLAMATH-PRECINCT 07 07', 'KLAMATH-PRECINCT 08': 'KLAMATH-PRECINCT 08 08', 'KLAMATH-PRECINCT 09': 'KLAMATH-PRECINCT 09 09', 'KLAMATH-PRECINCT 10': 'KLAMATH-PRECINCT 10 10', 'KLAMATH-PRECINCT 11': 'KLAMATH-PRECINCT 11 11', 'KLAMATH-PRECINCT 12': 'KLAMATH-PRECINCT 12 12', 'KLAMATH-PRECINCT 13': 'KLAMATH-PRECINCT 13 13', 'KLAMATH-PRECINCT 14': 'KLAMATH-PRECINCT 14 14', 'KLAMATH-PRECINCT 15': 'KLAMATH-PRECINCT 15 15', 'KLAMATH-PRECINCT 16': 'KLAMATH-PRECINCT 16 16', 'KLAMATH-PRECINCT 17': 'KLAMATH-PRECINCT 17 17', 'KLAMATH-PRECINCT 18': 'KLAMATH-PRECINCT 18 18', 'KLAMATH-PRECINCT 19': 'KLAMATH-PRECINCT 19 19', 'KLAMATH-PRECINCT 20': 'KLAMATH-PRECINCT 20 20', 'KLAMATH-PRECINCT 21': 'KLAMATH-PRECINCT 21 21', 'KLAMATH-PRECINCT 22': 'KLAMATH-PRECINCT 22 22', 'KLAMATH-PRECINCT 23': 'KLAMATH-PRECINCT 23 23', 'KLAMATH-PRECINCT 24': 'KLAMATH-PRECINCT 24 24', 'KLAMATH-PRECINCT 25': 'KLAMATH-PRECINCT 25 25', 'KLAMATH-PRECINCT 26': 'KLAMATH-PRECINCT 26 26', 'KLAMATH-PRECINCT 27': 'KLAMATH-PRECINCT 27 27', 'KLAMATH-PRECINCT 28': 'KLAMATH-PRECINCT 28 28', 'KLAMATH-PRECINCT 29': 'KLAMATH-PRECINCT 29 29', 'KLAMATH-PRECINCT 30': 'KLAMATH-PRECINCT 30 30', 'KLAMATH-PRECINCT 31': 'KLAMATH-PRECINCT 31 31', 'KLAMATH-PRECINCT 32': 'KLAMATH-PRECINCT 32 32', 'KLAMATH-PRECINCT 33': 'KLAMATH-PRECINCT 33 33', 'KLAMATH-PRECINCT 34': 'KLAMATH-PRECINCT 34 34', 'KLAMATH-PRECINCT 35': 'KLAMATH-PRECINCT 35 35', 'KLAMATH-PRECINCT 36': 'KLAMATH-PRECINCT 36 36', 'KLAMATH-PRECINCT 37': 'KLAMATH-PRECINCT 37 37', 'KLAMATH-PRECINCT 38': 'KLAMATH-PRECINCT 38 38', 'KLAMATH-PRECINCT 39': 'KLAMATH-PRECINCT 39 39', 'KLAMATH-PRECINCT 40': 'KLAMATH-PRECINCT 40 40', 'KLAMATH-PRECINCT 41': 'KLAMATH-PRECINCT 41 41', 'KLAMATH-PRECINCT 42': 'KLAMATH-PRECINCT 42 42', 'KLAMATH-PRECINCT 43': 'KLAMATH-PRECINCT 43 43', 'LAKE-CHRISTMAS VALLEY': 'LAKE-CHRISTMAS VALLEY 13', 'LAKE-COGSWELL CREEK': 'LAKE-COGSWELL CREEK 11', 'LAKE-CROOKED CREEK': 'LAKE-CROOKED CREEK 04', 'LAKE-FORT ROCK': 'LAKE-FORT ROCK 14', 'LAKE-FREMONT': 'LAKE-FREMONT 09', 'LAKE-GOOSE LAKE': 'LAKE-GOOSE LAKE 10', 'LAKE-NEW PINE CREEK': 'LAKE-NEW PINE CREEK 16', 'LAKE-NO. CENTRAL LKVW.': 'LAKE-N CENTRAL LAKEVIEW 20', 'LAKE-NORTH LAKEVIEW': 'LAKE-NORTH LAKEVIEW 07', 'LAKE-NORTH WARNER': 'LAKE-NORTH WARNER 05', 'LAKE-PAISLEY': 'LAKE-PAISLEY 03', 'LAKE-SILVER LAKE': 'LAKE-SILVER LAKE 01', 'LAKE-SO. CENTRAL LKVW.': 'LAKE-S CENTRAL LAKEVIEW 21', 'LAKE-SOUTH LAKEVIEW': 'LAKE-SOUTH LAKEVIEW 08', 'LAKE-SOUTH WARNER': 'LAKE-SOUTH WARNER 06', 'LAKE-SUMMER LAKE': 'LAKE-SUMMER LAKE 02', 'LAKE-THOM. CRK. WESTSIDE': 'LAKE-THOMAS CREEK 12', 'LAKE-VALLEY FALLS PREC.': 'LAKE-VALLEY FALLS 18', 'LAKE-WEST LAKEVIEW': 'LAKE-WEST LAKEVIEW 15', 'LANE-PRECINCT 702': 'LANE-702 702', 'LANE-PRECINCT 704': 'LANE-704 704', 'LANE-PRECINCT 706': 'LANE-706 706', 'LANE-PRECINCT 708': 'LANE-708 708', 'LANE-PRECINCT 710': 'LANE-710 710', 'LANE-PRECINCT 712': 'LANE-712 712', 'LANE-PRECINCT 714': 'LANE-714 714', 'LANE-PRECINCT 716': 'LANE-716 716', 'LANE-PRECINCT 718': 'LANE-718 718', 'LANE-PRECINCT 720': 'LANE-720 720', 'LANE-PRECINCT 722': 'LANE-722 722', 'LANE-PRECINCT 724': 'LANE-724 724', 'LANE-PRECINCT 726': 'LANE-726 726', 'LANE-PRECINCT 728': 'LANE-728 728', 'LANE-PRECINCT 730': 'LANE-730 730', 'LANE-PRECINCT 732': 'LANE-732 732', 'LANE-PRECINCT 734': 'LANE-734 734', 'LANE-PRECINCT 736': 'LANE-736 736', 'LANE-PRECINCT 738': 'LANE-738 738', 'LANE-PRECINCT 740': 'LANE-740 740', 'LANE-PRECINCT 742': 'LANE-742 742', 'LANE-PRECINCT 744': 'LANE-744 744', 'LANE-PRECINCT 746': 'LANE-746 746', 'LANE-PRECINCT 748': 'LANE-748 748', 'LANE-PRECINCT 750': 'LANE-750 750', 'LANE-PRECINCT 752': 'LANE-752 752', 'LANE-PRECINCT 754': 'LANE-754 754', 'LANE-PRECINCT 756': 'LANE-756 756', 'LANE-PRECINCT 758': 'LANE-758 758', 'LANE-PRECINCT 760': 'LANE-760 760', 'LANE-PRECINCT 762': 'LANE-762 762', 'LANE-PRECINCT 764': 'LANE-764 764', 'LANE-PRECINCT 766': 'LANE-766 766', 'LANE-PRECINCT 1121': 'LANE-1121 1121', 'LANE-PRECINCT 1123': 'LANE-1123 1123', 'LANE-PRECINCT 1125': 'LANE-1125 1125', 'LANE-PRECINCT 1127': 'LANE-1127 1127', 'LANE-PRECINCT 1231': 'LANE-1231 1231', 'LANE-PRECINCT 1233': 'LANE-1233 1233', 'LANE-PRECINCT 1235': 'LANE-1235 1235', 'LANE-PRECINCT 1341': 'LANE-1341 1341', 'LANE-PRECINCT 1343': 'LANE-1343 1343', 'LANE-PRECINCT 1345': 'LANE-1345 1345', 'LANE-PRECINCT 1349': 'LANE-1349 1349', 'LANE-PRECINCT 1451': 'LANE-1451 1451', 'LANE-PRECINCT 1453': 'LANE-1453 1453', 'LANE-PRECINCT 1455': 'LANE-1455 1455', 'LANE-PRECINCT 1457': 'LANE-1457 1457', 'LANE-PRECINCT 1561': 'LANE-1561 1561', 'LANE-PRECINCT 1563': 'LANE-1563 1563', 'LANE-PRECINCT 1565': 'LANE-1565 1565', 'LANE-PRECINCT 1569': 'LANE-1569 1569', 'LANE-PRECINCT 1671': 'LANE-1671 1671', 'LANE-PRECINCT 1673': 'LANE-1673 1673', 'LANE-PRECINCT 1675': 'LANE-1675 1675', 'LANE-PRECINCT 1781': 'LANE-1781 1781', 'LANE-PRECINCT 1783': 'LANE-1783 1783', 'LANE-PRECINCT 1785': 'LANE-1785 1785', 'LANE-PRECINCT 1787': 'LANE-1787 1787', 'LANE-PRECINCT 1891': 'LANE-1891 1891', 'LANE-PRECINCT 1893': 'LANE-1893 1893', 'LANE-PRECINCT 1895': 'LANE-1895 1895', 'LANE-PRECINCT 1897': 'LANE-1897 1897', 'LANE-PRECINCT 1899': 'LANE-1899 1899', 'LANE-PRECINCT 2122': 'LANE-2122 2122', 'LANE-PRECINCT 2234': 'LANE-2234 2234', 'LANE-PRECINCT 2238': 'LANE-2238 2238', 'LANE-PRECINCT 2340': 'LANE-2340 2340', 'LANE-PRECINCT 2344': 'LANE-2344 2344', 'LANE-PRECINCT 2456': 'LANE-2456 2456', 'LANE-PRECINCT 2562': 'LANE-2562 2562', 'LANE-PRECINCT 2676': 'LANE-2676 2676', 'LANE-PRECINCT 4200': 'LANE-4200 4200', 'LANE-PRECINCT 4301': 'LANE-4301 4301', 'LANE-PRECINCT 4400': 'LANE-4400 4400', 'LANE-PRECINCT 4500': 'LANE-4500 4500', 'LANE-PRECINCT 4601': 'LANE-4601 4601', 'LANE-PRECINCT 4700': 'LANE-4700 4700', 'LANE-PRECINCT 4800': 'LANE-4800 4800', 'LANE-PRECINCT 4900': 'LANE-4900 4900', 'LINCOLN-01 WALDPORT': 'LINCOLN-WALDPORT 01', 'LINCOLN-02 ALSEA': 'LINCOLN-ALLSEA 02', 'LINCOLN-03 SEAVIEW': 'LINCOLN-SEAVIEW 03', 'LINCOLN-04 TIDEWATER': 'LINCOLN-TIDEWATER 04', 'LINCOLN-05 YACHATS': 'LINCOLN-YACHATS 05', 'LINCOLN-06 BAYVIEW': 'LINCOLN-BAYVIEW 06', 'LINCOLN-07 SEAL ROCK': 'LINCOLN-SEAL ROCK 07', 'LINCOLN-08 SOUTH BEACH': 'LINCOLN-SOUTH BEACH 08', 'LINCOLN-09 NYE CREEK': 'LINCOLN-NYE CREEK 09', 'LINCOLN-10 NEWPORT BAY': 'LINCOLN-NEWPORT BAY 10', 'LINCOLN-11 OCEANVIEW': 'LINCOLN-OCEANVIEW 11', 'LINCOLN-12 YAQUINA': 'LINCOLN-YAQUINA 12', 'LINCOLN-13 PACIFIC': 'LINCOLN-PACIFIC 13', 'LINCOLN-14 AGATE BEACH': 'LINCOLN-AGATE BEACH 14', 'LINCOLN-15 OTTER ROCK': 'LINCOLN-OTTER ROCK 15', 'LINCOLN-16 DEPOE BAY': 'LINCOLN-DEPOE BAY 16', 'LINCOLN-17 FOGARTY CREEK': 'LINCOLN-FOGARTY CREEK 17', 'LINCOLN-18 KERN': 'LINCOLN-KERN 18', 'LINCOLN-19 SCHOONER CREEK': 'LINCOLN-SCHOONER CREEK 19', 'LINCOLN-20 DELAKE': 'LINCOLN-DELAKE 20', 'LINCOLN-21 OCEANLAKE': 'LINCOLN-OCEANLAKE 21', 'LINCOLN-22 SUNSET WEST': 'LINCOLN-SUNSET WEST 22', 'LINCOLN-23 SUNSET EAST': 'LINCOLN-SUNSET EAST 23', 'LINCOLN-24 ROSE LODGE': 'LINCOLN-ROSE LODGE 24', 'LINCOLN-25 BIG ELK': 'LINCOLN-BIG ELK 25', 'LINCOLN-26 ELK CITY': 'LINCOLN-ELK CITY 26', 'LINCOLN-27 FRUITVALE': 'LINCOLN-FRUITVALE 27', 'LINCOLN-28 EDDYVILLE': 'LINCOLN-EDDYVILLE 28', 'LINCOLN-29 SILETZ': 'LINCOLN-SILETZ 29', 'LINCOLN-30 ROCK CREEK': 'LINCOLN-ROCK CREEK 30', 'LINCOLN-31 EAST TOLEDO': 'LINCOLN-EAST TOLEDO 31', 'LINCOLN-32 SOUTH TOLEDO': 'LINCOLN-SOUTH TOLEDO 32', 'LINN-PRECINCT 001': 'LINN-PRECINCT 001 001', 'LINN-PRECINCT 003': 'LINN-PRECINCT 003 003', 'LINN-PRECINCT 005': 'LINN-PRECINCT 005 005', 'LINN-PRECINCT 008': 'LINN-PRECINCT 008 008', 'LINN-PRECINCT 011': 'LINN-PRECINCT 011 011', 'LINN-PRECINCT 014': 'LINN-PRECINCT 014 014', 'LINN-PRECINCT 015': 'LINN-PRECINCT 015 015', 'LINN-PRECINCT 016': 'LINN-PRECINCT 016 016', 'LINN-PRECINCT 018': 'LINN-PRECINCT 018 018', 'LINN-PRECINCT 019': 'LINN-PRECINCT 019 019', 'LINN-PRECINCT 020': 'LINN-PRECINCT 020 020', 'LINN-PRECINCT 021': 'LINN-PRECINCT 021 021', 'LINN-PRECINCT 022': 'LINN-PRECINCT 022 022', 'LINN-PRECINCT 023': 'LINN-PRECINCT 023 023', 'LINN-PRECINCT 025': 'LINN-PRECINCT 025 025', 'LINN-PRECINCT 026': 'LINN-PRECINCT 026 026', 'LINN-PRECINCT 027': 'LINN-PRECINCT 027 027', 'LINN-PRECINCT 028': 'LINN-PRECINCT 028 028', 'LINN-PRECINCT 029': 'LINN-PRECINCT 029 029', 'LINN-PRECINCT 030': 'LINN-PRECINCT 030 030', 'LINN-PRECINCT 031': 'LINN-PRECINCT 031 031', 'LINN-PRECINCT 032': 'LINN-PRECINCT 032 032', 'LINN-PRECINCT 033': 'LINN-PRECINCT 033 033', 'LINN-PRECINCT 034': 'LINN-PRECINCT 034 034', 'LINN-PRECINCT 035': 'LINN-PRECINCT 035 035', 'LINN-PRECINCT 036': 'LINN-PRECINCT 036 036', 'LINN-PRECINCT 037': 'LINN-PRECINCT 037 037', 'LINN-PRECINCT 038': 'LINN-PRECINCT 038 038', 'LINN-PRECINCT 039': 'LINN-PRECINCT 039 039', 'LINN-PRECINCT 041': 'LINN-PRECINCT 041 041', 'LINN-PRECINCT 042': 'LINN-PRECINCT 042 042', 'LINN-PRECINCT 043': 'LINN-PRECINCT 043 043', 'LINN-PRECINCT 044': 'LINN-PRECINCT 044 044', 'LINN-PRECINCT 045': 'LINN-PRECINCT 045 045', 'LINN-PRECINCT 046': 'LINN-PRECINCT 046 046', 'LINN-PRECINCT 047': 'LINN-PRECINCT 047 047', 'LINN-PRECINCT 048': 'LINN-PRECINCT 048 048', 'LINN-PRECINCT 049': 'LINN-PRECINCT 049 049', 'LINN-PRECINCT 050': 'LINN-PRECINCT 050 050', 'LINN-PRECINCT 051': 'LINN-PRECINCT 051 051', 'LINN-PRECINCT 052': 'LINN-PRECINCT 052 052', 'LINN-PRECINCT 053': 'LINN-PRECINCT 053 053', 'LINN-PRECINCT 054': 'LINN-PRECINCT 054 054', 'LINN-PRECINCT 055': 'LINN-PRECINCT 055 055', 'LINN-PRECINCT 056': 'LINN-PRECINCT 056 056', 'LINN-PRECINCT 057': 'LINN-PRECINCT 057 057', 'LINN-PRECINCT 058': 'LINN-PRECINCT 058 058', 'LINN-PRECINCT 059': 'LINN-PRECINCT 059 059', 'LINN-PRECINCT 060': 'LINN-PRECINCT 060 060', 'LINN-PRECINCT 061': 'LINN-PRECINCT 061 061', 'LINN-PRECINCT 062': 'LINN-PRECINCT 062 062', 'LINN-PRECINCT 063': 'LINN-PRECINCT 063 063', 'LINN-PRECINCT 065': 'LINN-PRECINCT 065 065', 'LINN-PRECINCT 066': 'LINN-PRECINCT 066 066', 'LINN-PRECINCT 067': 'LINN-PRECINCT 067 067', 'LINN-PRECINCT 068': 'LINN-PRECINCT 068 068', 'LINN-PRECINCT 070': 'LINN-PRECINCT 070 070', 'LINN-PRECINCT 071': 'LINN-PRECINCT 071 071', 'LINN-PRECINCT 073': 'LINN-PRECINCT 073 073', 'LINN-PRECINCT 074': 'LINN-PRECINCT 074 074', 'LINN-PRECINCT 075': 'LINN-PRECINCT 075 075', 'LINN-PRECINCT 076': 'LINN-PRECINCT 076 076', 'LINN-PRECINCT 077': 'LINN-PRECINCT 077 077', 'LINN-PRECINCT 078': 'LINN-PRECINCT 078 078', 'LINN-PRECINCT 079': 'LINN-PRECINCT 079 079', 'LINN-PRECINCT 080': 'LINN-PRECINCT 080 080', 'LINN-PRECINCT 082': 'LINN-PRECINCT 082 082', 'LINN-PRECINCT 083': 'LINN-PRECINCT 083 083', 'LINN-PRECINCT 084': 'LINN-PRECINCT 084 084', 'LINN-PRECINCT 086': 'LINN-PRECINCT 086 086', 'LINN-PRECINCT 087': 'LINN-PRECINCT 087 087', 'LINN-PRECINCT 088': 'LINN-PRECINCT 088 088', 'LINN-PRECINCT 089': 'LINN-PRECINCT 089 089', 'LINN-PRECINCT 091': 'LINN-PRECINCT 091 091', 'LINN-PRECINCT 092': 'LINN-PRECINCT 092 092', 'LINN-PRECINCT 093': 'LINN-PRECINCT 093 093', 'LINN-PRECINCT 094': 'LINN-PRECINCT 094 094', 'LINN-PRECINCT 095': 'LINN-PRECINCT 095 095', 'LINN-PRECINCT 096': 'LINN-PRECINCT 096 096', 'LINN-PRECINCT 097': 'LINN-PRECINCT 097 097', 'LINN-PRECINCT 099': 'LINN-PRECINCT 099 099', 'LINN-PRECINCT 100': 'LINN-PRECINCT 100 100', 'LINN-PRECINCT 101': 'LINN-PRECINCT 101 101', 'LINN-PRECINCT 102': 'LINN-PRECINCT 102 102', 'LINN-PRECINCT 103': 'LINN-PRECINCT 103 103', 'LINN-PRECINCT 104': 'LINN-PRECINCT 104 104', 'LINN-PRECINCT 105': 'LINN-PRECINCT 105 105', 'LINN-PRECINCT 106': 'LINN-PRECINCT 106 106', 'LINN-PRECINCT 107': 'LINN-PRECINCT 107 107', 'LINN-PRECINCT 108': 'LINN-PRECINCT 108 108', 'MALHEUR-1 - ONTARIO 1': 'MALHEUR-ONTARIO 01', 'MALHEUR-2 - ONTARIO 2': 'MALHEUR-ONTARIO 02', 'MALHEUR-3 - ONTARIO 3': 'MALHEUR-ONTARIO 03', 'MALHEUR-4 - ONTARIO 4': 'MALHEUR-ONTARIO 04', 'MALHEUR-5 - APPLEGATE': 'MALHEUR-APPLEGATE 05', 'MALHEUR-6 - FAIR/ONTARIO HEIGHTS': 'MALHEUR-FAIR/ONTARIO HEIGHTS 06', 'MALHEUR-7 - BUTTE/CAIRO': 'MALHEUR-BUTTE/CAIRO 07', 'MALHEUR-8 - CITY OF NYSSA': 'MALHEUR-NYSSA CITY LIMITS 08', 'MALHEUR-9 - RURAL NYSSA': 'MALHEUR-RURAL NYSSA 09', 'MALHEUR-10 - CITY OF ADRIAN': 'MALHEUR-CITY OF ADRIAN 10', 'MALHEUR-11 - RURAL ADRIAN': 'MALHEUR-RURAL ADRIAN 11', 'MALHEUR-12 - CITY OF VALE': 'MALHEUR-VALE CITY LIMITS 12', 'MALHEUR-13 - SOUTH RURAL VALE': 'MALHEUR-SOUTH RURAL VALE 13', 'MALHEUR-14 - NORTH RURAL VALE': 'MALHEUR-NORTH RURAL VALE 14', 'MALHEUR-15 - WILLOWCREEK': 'MALHEUR-WILLOWCREEK 15', 'MALHEUR-16 - BROGAN': 'MALHEUR-BROGAN 16', 'MALHEUR-17 - IRONSIDE': 'MALHEUR-IRONSIDE 17', 'MALHEUR-18 - HARPER': 'MALHEUR-HARPER 18', 'MALHEUR-19 - JUNTURA': 'MALHEUR-JUNTURA 19', 'MALHEUR-20 - CITY OF JORDAN VALLEY': 'MALHEUR-CITY OF JORDAN VALLEY 20', 'MALHEUR-21 - RURAL JORDAN VALLEY': 'MALHEUR-RURAL JORDAN VALLEY 21', 'MALHEUR-22 - AROCK': 'MALHEUR-AROCK 22', 'MALHEUR-23 - MCDERMITT': 'MALHEUR-MCDERMITT 23', 'MALHEUR-24 - ANNEX/HUNTINGTON/FAREWELL BEND': 'MALHEUR-ANNEX/HUNTINGTON/FAREWELL BEND 24', 'MARION-310': 'MARION-310 310', 'MARION-311': 'MARION-311 311', 'MARION-312': 'MARION-312 312', 'MARION-320': 'MARION-320 320', 'MARION-321': 'MARION-321 321', 'MARION-322': 'MARION-322 322', 'MARION-323': 'MARION-323 323', 'MARION-324': 'MARION-324 324', 'MARION-325': 'MARION-325 325', 'MARION-326': 'MARION-326 326', 'MARION-330': 'MARION-330 330', 'MARION-331': 'MARION-331 331', 'MARION-332': 'MARION-332 332', 'MARION-333': 'MARION-333 333', 'MARION-334': 'MARION-334 334', 'MARION-335': 'MARION-335 335', 'MARION-336': 'MARION-336 336', 'MARION-337': 'MARION-337 337', 'MARION-340': 'MARION-340 340', 'MARION-341': 'MARION-341 341', 'MARION-342': 'MARION-342 342', 'MARION-343': 'MARION-343 343', 'MARION-344': 'MARION-344 344', 'MARION-350': 'MARION-350 350', 'MARION-351': 'MARION-351 351', 'MARION-352': 'MARION-352 352', 'MARION-353': 'MARION-353 353', 'MARION-354': 'MARION-354 354', 'MARION-355': 'MARION-355 355', 'MARION-356': 'MARION-356 356', 'MARION-357': 'MARION-357 357', 'MARION-361': 'MARION-361 361', 'MARION-362': 'MARION-362 362', 'MARION-363': 'MARION-363 363', 'MARION-364': 'MARION-364 364', 'MARION-365': 'MARION-365 365', 'MARION-370': 'MARION-370 370', 'MARION-371': 'MARION-371 371', 'MARION-372': 'MARION-372 372', 'MARION-373': 'MARION-373 373', 'MARION-374': 'MARION-374 374', 'MARION-375': 'MARION-375 375', 'MARION-376': 'MARION-376 376', 'MARION-400': 'MARION-400 400', 'MARION-401': 'MARION-401 401', 'MARION-402': 'MARION-402 402', 'MARION-403': 'MARION-403 403', 'MARION-404': 'MARION-404 404', 'MARION-405': 'MARION-405 405', 'MARION-406': 'MARION-406 406', 'MARION-407': 'MARION-407 407', 'MARION-408': 'MARION-408 408', 'MARION-505': 'MARION-505 505', 'MARION-515': 'MARION-515 515', 'MARION-525': 'MARION-525 525', 'MARION-535': 'MARION-535 535', 'MARION-545': 'MARION-545 545', 'MARION-555': 'MARION-555 555', 'MARION-577': 'MARION-577 577', 'MARION-578': 'MARION-578 578', 'MARION-579': 'MARION-579 579', 'MARION-580': 'MARION-580 580', 'MARION-581': 'MARION-581 581', 'MARION-582': 'MARION-582 582', 'MARION-583': 'MARION-583 583', 'MARION-584': 'MARION-584 584', 'MARION-586': 'MARION-586 586', 'MARION-587': 'MARION-587 587', 'MARION-588': 'MARION-588 588', 'MARION-589': 'MARION-589 589', 'MARION-590': 'MARION-590 590', 'MARION-615': 'MARION-615 615', 'MARION-625': 'MARION-625 625', 'MARION-635': 'MARION-635 635', 'MARION-645': 'MARION-645 645', 'MARION-655': 'MARION-655 655', 'MARION-674': 'MARION-674 674', 'MARION-676': 'MARION-676 676', 'MARION-677': 'MARION-677 677', 'MARION-678': 'MARION-678 678', 'MARION-679': 'MARION-679 679', 'MARION-680': 'MARION-680 680', 'MARION-681': 'MARION-681 681', 'MARION-682': 'MARION-682 682', 'MARION-683': 'MARION-683 683', 'MARION-684': 'MARION-684 684', 'MARION-686': 'MARION-686 686', 'MARION-687': 'MARION-687 687', 'MARION-688': 'MARION-688 688', 'MARION-689': 'MARION-689 689', 'MARION-690': 'MARION-690 690', 'MARION-691': 'MARION-691 691', 'MARION-692': 'MARION-692 692', 'MARION-694': 'MARION-694 694', 'MARION-696': 'MARION-696 696', 'MARION-697': 'MARION-697 697', 'MARION-698': 'MARION-698 698', 'MARION-705': 'MARION-705 705', 'MARION-715': 'MARION-715 715', 'MARION-725': 'MARION-725 725', 'MARION-779': 'MARION-779 779', 'MARION-780': 'MARION-780 780', 'MARION-781': 'MARION-781 781', 'MARION-782': 'MARION-782 782', 'MARION-783': 'MARION-783 783', 'MARION-784': 'MARION-784 784', 'MARION-786': 'MARION-786 786', 'MARION-787': 'MARION-787 787', 'MARION-788': 'MARION-788 788', 'MARION-789': 'MARION-789 789', 'MARION-790': 'MARION-790 790', 'MARION-815': 'MARION-815 815', 'MARION-825': 'MARION-825 825', 'MARION-835': 'MARION-835 835', 'MARION-845': 'MARION-845 845', 'MARION-855': 'MARION-855 855', 'MARION-865': 'MARION-865 865', 'MARION-905': 'MARION-905 905', 'MARION-915': 'MARION-915 915', 'MARION-925': 'MARION-925 925', 'MARION-935': 'MARION-935 935', 'MARION-981': 'MARION-981 981', 'MARION-982': 'MARION-982 982', 'MORROW-PRECINCT 1 - BOARDMAN': 'MORROW-BOARDMAN 01', 'MORROW-PRECINCT 2 - IRRIGON': 'MORROW-IRRIGON 02', 'MORROW-PRECINCT 3 - LEXINGTON': 'MORROW-LEXINGTON 03', 'MORROW-PRECINCT 4 - IONE': 'MORROW-IONE 04', 'MORROW-PRECINCT 5 - HEPPNER/HARDMAN': 'MORROW-HEPPNER/HARDMAN 05', 'MULTNOMAH-2701': 'MULTNOMAH-2701 2701', 'MULTNOMAH-2702': 'MULTNOMAH-2702 2702', 'MULTNOMAH-3101': 'MULTNOMAH-3101 3101', 'MULTNOMAH-3102': 'MULTNOMAH-3102 3102', 'MULTNOMAH-3103': 'MULTNOMAH-3103 3103', 'MULTNOMAH-3301': 'MULTNOMAH-3301 3301', 'MULTNOMAH-3303': 'MULTNOMAH-3303 3303', 'MULTNOMAH-3304': 'MULTNOMAH-3304 3304', 'MULTNOMAH-3501': 'MULTNOMAH-3501 3501', 'MULTNOMAH-3502': 'MULTNOMAH-3502 3502', 'MULTNOMAH-3602': 'MULTNOMAH-3602 3602', 'MULTNOMAH-3603': 'MULTNOMAH-3603 3603', 'MULTNOMAH-3604': 'MULTNOMAH-3604 3604', 'MULTNOMAH-3605': 'MULTNOMAH-3605 3605', 'MULTNOMAH-3606': 'MULTNOMAH-3606 3606', 'MULTNOMAH-3607': 'MULTNOMAH-3607 3607', 'MULTNOMAH-3608': 'MULTNOMAH-3608 3608', 'MULTNOMAH-3609': 'MULTNOMAH-3609 3609', 'MULTNOMAH-3610': 'MULTNOMAH-3610 3610', 'MULTNOMAH-3611': 'MULTNOMAH-3611 3611', 'MULTNOMAH-3612': 'MULTNOMAH-3612 3612', 'MULTNOMAH-3801': 'MULTNOMAH-3801 3801', 'MULTNOMAH-3802': 'MULTNOMAH-3802 3802', 'MULTNOMAH-3803': 'MULTNOMAH-3803 3803', 'MULTNOMAH-3804': 'MULTNOMAH-3804 3804', 'MULTNOMAH-3805': 'MULTNOMAH-3805 3805', 'MULTNOMAH-3806': 'MULTNOMAH-3806 3806', 'MULTNOMAH-3807': 'MULTNOMAH-3807 3807', 'MULTNOMAH-3808': 'MULTNOMAH-3808 3808', 'MULTNOMAH-4101': 'MULTNOMAH-4101 4101', 'MULTNOMAH-4102': 'MULTNOMAH-4102 4102', 'MULTNOMAH-4103': 'MULTNOMAH-4103 4103', 'MULTNOMAH-4203': 'MULTNOMAH-4203 4203', 'MULTNOMAH-4204': 'MULTNOMAH-4204 4204', 'MULTNOMAH-4205': 'MULTNOMAH-4205 4205', 'MULTNOMAH-4207': 'MULTNOMAH-4207 4207', 'MULTNOMAH-4208': 'MULTNOMAH-4208 4208', 'MULTNOMAH-4209': 'MULTNOMAH-4209 4209', 'MULTNOMAH-4210': 'MULTNOMAH-4210 4210', 'MULTNOMAH-4301': 'MULTNOMAH-4301 4301', 'MULTNOMAH-4302': 'MULTNOMAH-4302 4302', 'MULTNOMAH-4304': 'MULTNOMAH-4304 4304', 'MULTNOMAH-4305': 'MULTNOMAH-4305 4305', 'MULTNOMAH-4306': 'MULTNOMAH-4306 4306', 'MULTNOMAH-4310': 'MULTNOMAH-4310 4310', 'MULTNOMAH-4401': 'MULTNOMAH-4401 4401', 'MULTNOMAH-4402': 'MULTNOMAH-4402 4402', 'MULTNOMAH-4403': 'MULTNOMAH-4403 4403', 'MULTNOMAH-4404': 'MULTNOMAH-4404 4404', 'MULTNOMAH-4409': 'MULTNOMAH-4409 4409', 'MULTNOMAH-4501': 'MULTNOMAH-4501 4501', 'MULTNOMAH-4502': 'MULTNOMAH-4502 4502', 'MULTNOMAH-4503': 'MULTNOMAH-4503 4503', 'MULTNOMAH-4505': 'MULTNOMAH-4505 4505', 'MULTNOMAH-4506': 'MULTNOMAH-4506 4506', 'MULTNOMAH-4508': 'MULTNOMAH-4508 4508', 'MULTNOMAH-4509': 'MULTNOMAH-4509 4509', 'MULTNOMAH-4510': 'MULTNOMAH-4510 4510', 'MULTNOMAH-4511': 'MULTNOMAH-4511 4511', 'MULTNOMAH-4512': 'MULTNOMAH-4512 4512', 'MULTNOMAH-4601': 'MULTNOMAH-4601 4601', 'MULTNOMAH-4602': 'MULTNOMAH-4602 4602', 'MULTNOMAH-4604': 'MULTNOMAH-4604 4604', 'MULTNOMAH-4605': 'MULTNOMAH-4605 4605', 'MULTNOMAH-4606': 'MULTNOMAH-4606 4606', 'MULTNOMAH-4607': 'MULTNOMAH-4607 4607', 'MULTNOMAH-4608': 'MULTNOMAH-4608 4608', 'MULTNOMAH-4701': 'MULTNOMAH-4701 4701', 'MULTNOMAH-4702': 'MULTNOMAH-4702 4702', 'MULTNOMAH-4703': 'MULTNOMAH-4703 4703', 'MULTNOMAH-4704': 'MULTNOMAH-4704 4704', 'MULTNOMAH-4705': 'MULTNOMAH-4705 4705', 'MULTNOMAH-4706': 'MULTNOMAH-4706 4706', 'MULTNOMAH-4707': 'MULTNOMAH-4707 4707', 'MULTNOMAH-4708': 'MULTNOMAH-4708 4708', 'MULTNOMAH-4709': 'MULTNOMAH-4709 4709', 'MULTNOMAH-4710': 'MULTNOMAH-4710 4710', 'MULTNOMAH-4801': 'MULTNOMAH-4801 4801', 'MULTNOMAH-4802': 'MULTNOMAH-4802 4802', 'MULTNOMAH-4803': 'MULTNOMAH-4803 4803', 'MULTNOMAH-4804': 'MULTNOMAH-4804 4804', 'MULTNOMAH-4805': 'MULTNOMAH-4805 4805', 'MULTNOMAH-4806': 'MULTNOMAH-4806 4806', 'MULTNOMAH-4807': 'MULTNOMAH-4807 4807', 'MULTNOMAH-4901': 'MULTNOMAH-4901 4901', 'MULTNOMAH-4902': 'MULTNOMAH-4902 4902', 'MULTNOMAH-4903': 'MULTNOMAH-4903 4903', 'MULTNOMAH-4904': 'MULTNOMAH-4904 4904', 'MULTNOMAH-4905': 'MULTNOMAH-4905 4905', 'MULTNOMAH-4906': 'MULTNOMAH-4906 4906', 'MULTNOMAH-4907': 'MULTNOMAH-4907 4907', 'MULTNOMAH-4908': 'MULTNOMAH-4908 4908', 'MULTNOMAH-4909': 'MULTNOMAH-4909 4909', 'MULTNOMAH-4910': 'MULTNOMAH-4910 4910', 'MULTNOMAH-4911': 'MULTNOMAH-4911 4911', 'MULTNOMAH-5001': 'MULTNOMAH-5001 5001', 'MULTNOMAH-5002': 'MULTNOMAH-5002 5002', 'MULTNOMAH-5003': 'MULTNOMAH-5003 5003', 'MULTNOMAH-5004': 'MULTNOMAH-5004 5004', 'MULTNOMAH-5006': 'MULTNOMAH-5006 5006', 'MULTNOMAH-5007': 'MULTNOMAH-5007 5007', 'MULTNOMAH-5008': 'MULTNOMAH-5008 5008', 'MULTNOMAH-5009': 'MULTNOMAH-5009 5009', 'MULTNOMAH-5101': 'MULTNOMAH-5101 5101', 'MULTNOMAH-5102': 'MULTNOMAH-5102 5102', 'MULTNOMAH-5103': 'MULTNOMAH-5103 5103', 'MULTNOMAH-5104': 'MULTNOMAH-5104 5104', 'MULTNOMAH-5105': 'MULTNOMAH-5105 5105', 'MULTNOMAH-5106': 'MULTNOMAH-5106 5106', 'MULTNOMAH-5107': 'MULTNOMAH-5107 5107', 'MULTNOMAH-5202': 'MULTNOMAH-5202 5202', 'MULTNOMAH-5203': 'MULTNOMAH-5203 5203', 'MULTNOMAH-5204': 'MULTNOMAH-5204 5204', 'POLK-102': 'POLK-POLK 102 102', 'POLK-106': 'POLK-POLK 106 106', 'POLK-110': 'POLK-POLK 110 110', 'POLK-118': 'POLK-POLK 118 118', 'POLK-122': 'POLK-POLK 122 122', 'POLK-134': 'POLK-POLK 134 134', 'POLK-136': 'POLK-POLK 136 136', 'POLK-140': 'POLK-POLK 140 140', 'POLK-146': 'POLK-POLK 146 146', 'POLK-152': 'POLK-POLK 152 152', 'POLK-154': 'POLK-POLK 154 154', 'POLK-156': 'POLK-POLK 156 156', 'POLK-162': 'POLK-POLK 162 162', 'POLK-166': 'POLK-POLK 166 166', 'POLK-178': 'POLK-POLK 178 178', 'POLK-182': 'POLK-POLK 182 182', 'SHERMAN-1 RUFUS': 'SHERMAN-RUFUS 01', 'SHERMAN-2 WASCO': 'SHERMAN-WASCO 02', 'SHERMAN-3 MORO': 'SHERMAN-MORO 03', 'SHERMAN-4 GRASS VALLEY': 'SHERMAN-GRASS VALLEY 04', 'SHERMAN-5 KENT': 'SHERMAN-KENT 05', 'TILLAMOOK-BAY': 'TILLAMOOK-BAY 001', 'TILLAMOOK-BEAVER': 'TILLAMOOK-BEAVER 003', 'TILLAMOOK-CARNAHAN': 'TILLAMOOK-CARNAHAN 005', 'TILLAMOOK-CLOVERDALE': 'TILLAMOOK-CLOVERDALE 006', 'TILLAMOOK-EASTSIDE': 'TILLAMOOK-EASTSIDE 007', 'TILLAMOOK-FAIRVIEW': 'TILLAMOOK-FAIRVIEW 008', 'TILLAMOOK-FOLEY': 'TILLAMOOK-FOLEY 010', 'TILLAMOOK-GARIBALDI': 'TILLAMOOK-GARIBALDI 012', 'TILLAMOOK-HEBO': 'TILLAMOOK-HEBO 013', 'TILLAMOOK-KILCHIS': 'TILLAMOOK-KILCHIS 014', 'TILLAMOOK-MAPLELEAF': 'TILLAMOOK-MAPLELEAF 015', 'TILLAMOOK-NEHALEM': 'TILLAMOOK-NEHALEM 016', 'TILLAMOOK-NESKOWIN': 'TILLAMOOK-NESKOWIN 017', 'TILLAMOOK-NETARTS': 'TILLAMOOK-NETARTS 018', 'TILLAMOOK-OCEANSIDE': 'TILLAMOOK-OCEANSIDE 020', 'TILLAMOOK-PINE GROVE': 'TILLAMOOK-PINE GROVE 021', 'TILLAMOOK-ROCKAWAY': 'TILLAMOOK-ROCKAWAY 024', 'TILLAMOOK-SOUTH PRAIRIE EAST': 'TILLAMOOK-SOUTH PRAIRIE EAST 027', 'TILLAMOOK-SOUTH PRAIRIE WEST': 'TILLAMOOK-SOUTH PRAIRIE WEST 026', 'TILLAMOOK-TRASK': 'TILLAMOOK-TRASK 028', 'TILLAMOOK-UNION': 'TILLAMOOK-UNION 030', 'TILLAMOOK-WESTSIDE': 'TILLAMOOK-WESTSIDE 032', 'TILLAMOOK-WHEELER': 'TILLAMOOK-WHEELER 033', 'TILLAMOOK-TILLAMOOK #1': 'TILLAMOOK-TILLAMOOK #1 035', 'TILLAMOOK-TILLAMOOK #2': 'TILLAMOOK-TILLAMOOK #2 036', 'TILLAMOOK-TILLAMOOK #3': 'TILLAMOOK-TILLAMOOK #3 037', 'TILLAMOOK-TILLAMOOK #4': 'TILLAMOOK-TILLAMOOK #4 038', 'TILLAMOOK-TILLAMOOK #5': 'TILLAMOOK-TILLAMOOK #5 039', 'TILLAMOOK-TILLAMOOK #6': 'TILLAMOOK-TILLAMOOK #6 040', 'UMATILLA-101-UMATILLA,OUTSIDE CITY': 'UMATILLA-UMATILLA OUTSIDE 101', 'UMATILLA-102-UMATILLA,CITY OF': 'UMATILLA-UMATILLA CITY 102', 'UMATILLA-103-HERMISTON,OUTSIDE': 'UMATILLA-HERMISTON OUTSIDE 103', 'UMATILLA-104-HERMISTON,OUTSIDE': 'UMATILLA-HERMISTON OUTSIDE 104', 'UMATILLA-105-HERMISTON,OUTSIDE': 'UMATILLA-HERMISTON OUTSIDE 105', 'UMATILLA-110-PENDLETON,OUTSIDE': 'UMATILLA-PENDLETON OUTSIDE 110', 'UMATILLA-111-PENDLETON,OUTSIDE': 'UMATILLA-PENDLETON OUTSIDE 111', 'UMATILLA-112-MEACHAM': 'UMATILLA-MEACHAM 112', 'UMATILLA-113-PILOT ROCK,CITY OF': 'UMATILLA-PILOT ROCK CITY 113', 'UMATILLA-114-PILOT ROCK,OUTSIDE': 'UMATILLA-PILOT ROCK OUTSIDE 114', 'UMATILLA-115-PENDLETON,OUTSIDE': 'UMATILLA-PENDLETON OUTSIDE 115', 'UMATILLA-116-ATHENA,OUTSIDE CITY': 'UMATILLA-ATHENA OUTSIDE 116', 'UMATILLA-119-UMAPINE': 'UMATILLA-UMAPINE 119', 'UMATILLA-120-MILTON-FRWTR,OUTSIDE': 'UMATILLA-MILTON-FREEWATER OUTSIDE 120', 'UMATILLA-121-MILTON-FRWTR,OUTSIDE': 'UMATILLA-MILTON-FREEWATER OUTSIDE 121', 'UMATILLA-123-MILTON-FRWTR,OUTSIDE': 'UMATILLA-MILTON-FREEWATER OUTSIDE 123', 'UMATILLA-126-HERMISTON,CITY OF': 'UMATILLA-HERMISTON CITY 126', 'UMATILLA-127-HERMISTON,CITY OF': 'UMATILLA-HERMISTON CITY 127', 'UMATILLA-128-HERMISTON,CITY OF': 'UMATILLA-HERMISTON CITY 128', 'UMATILLA-129-HERMISTON,CITY OF': 'UMATILLA-HERMISTON CITY 129', 'UMATILLA-130-HERMISTON,CITY OF': 'UMATILLA-HERMISTON CITY 130', 'UMATILLA-131-MILTON-FRWTR,CITY OF': 'UMATILLA-MILTON-FREEWATER CITY 131', 'UMATILLA-132-MILTON-FRWTR,CITY OF': 'UMATILLA-MILTON-FREEWATER CITY 132', 'UMATILLA-133-MILTON-FRWTR,CITY OF': 'UMATILLA-MILTON-FREEWATER CITY 133', 'UMATILLA-136-PENDLETON,CITY OF': 'UMATILLA-PENDLETON CITY 136', 'UMATILLA-137-PENDLETON,CITY OF': 'UMATILLA-PENDLETON CITY 137', 'UMATILLA-138-PENDLETON,CITY OF': 'UMATILLA-PENDLETON CITY 138', 'UMATILLA-139-PENDLETON,CITY OF': 'UMATILLA-PENDLETON CITY 139', 'UMATILLA-141-PENDLETON,CITY OF': 'UMATILLA-PENDLETON CITY 141', 'UMATILLA-142-PENDLETON,CITY OF': 'UMATILLA-PENDLETON CITY 142', 'UMATILLA-143-PENDLETON,CITY OF': 'UMATILLA-PENDLETON CITY 143', 'UMATILLA-106C-STANFIELD,CITY OF': 'UMATILLA-STANFIELD CITY 106C', 'UMATILLA-106R-STANFIELD,OUTSIDE': 'UMATILLA-STANFIELD OUTSIDE 106R', 'UMATILLA-107C-ECHO,CITY OF': 'UMATILLA-ECHO CITY 107C', 'UMATILLA-107R-ECHO,OUTSIDE CITY': 'UMATILLA-ECHO OUTSIDE 107R', 'UMATILLA-108C-HELIX,CITY OF': 'UMATILLA-HELIX CITY 108C', 'UMATILLA-108R-HELIX,OUTSIDE CITY': 'UMATILLA-HELIX OUTSIDE 108R', 'UMATILLA-109C-ADAMS,CITY OF': 'UMATILLA-ADAMS CITY 109C', 'UMATILLA-109R-ADAMS,OUTSIDE CITY': 'UMATILLA-ADAMS OUTSIDE 109R', 'UMATILLA-117C-UKIAH,CITY OF': 'UMATILLA-UKIAH CITY 117C', 'UMATILLA-117R-UKIAH,OUTSIDE CITY': 'UMATILLA-UKIAH OUTSIDE 117R', 'UMATILLA-124C-ATHENA,CITY OF': 'UMATILLA-ATHENA CITY 124C', 'UMATILLA-124R-ATHENA,OUTSIDE CITY': 'UMATILLA-ATHENA OUTSIDE 124R', 'UMATILLA-125C-WESTON,CITY OF': 'UMATILLA-WESTON CITY 125C', 'UMATILLA-125R-WESTON,OUTSIDE CITY': 'UMATILLA-WESTON OUTSIDE 125R', 'UNION-01 LG CITY': 'UNION-LA GRANDE CITY 1 01', 'UNION-02-LG CITY': 'UNION-LA GRANDE CITY 2 02', 'UNION-03-LG CITY': 'UNION-LA GRANDE CITY 3 03', 'UNION-04-LG RURAL': 'UNION-LA GRANDE RURAL 4 04', 'UNION-05-LG RURAL': 'UNION-LA GRANDE RURAL 5 05', 'UNION-06-COVE CITY': 'UNION-COVE CITY 06', 'UNION-07-COVE RURAL': 'UNION-COVE RURAL 07', 'UNION-08-ELGIN CITY': 'UNION-ELGIN CITY 08', 'UNION-09-ELGIN RURAL': 'UNION-ELGIN RURAL 09', 'UNION-10-IMB CITY': 'UNION-IMBLER CITY 10', 'UNION-11-IMB/SUM RURAL': 'UNION-IMBLER/SUMMERVILLE RURAL 11', 'UNION-12-SUM CITY': 'UNION-SUMMERVILLE CITY 12', 'UNION-13-NP RURAL': 'UNION-NORTH POWDER RURAL 13', 'UNION-14-NP CITY': 'UNION-NORTH POWDER CITY 14', 'UNION-15-UNION RURAL': 'UNION-UNION RURAL 15', 'UNION-16-UNION CITY': 'UNION-UNION CITY 16', 'UNION-17-ANT/BC': 'UNION-ANTELOPE/BIG CREEK 17', 'UNION-18-IC CITY': 'UNION-ISLAND CITY 18', 'UNION-19-IC RURAL': 'UNION-ISLAND CITY RURAL 19', 'WALLOWA-1': 'WALLOWA-ENTERPRISE #1 01', 'WALLOWA-2': 'WALLOWA-ENTERPRISE #2 02', 'WALLOWA-3': 'WALLOWA-ENTERPRISE #3 03', 'WALLOWA-4': 'WALLOWA-ENTERPRISE #4 04', 'WALLOWA-5': 'WALLOWA-JOSEPH #5 05', 'WALLOWA-6': 'WALLOWA-JOSEPH #6 06', 'WALLOWA-7': 'WALLOWA-WALLOWA #7 07', 'WALLOWA-8': 'WALLOWA-WALLOWA #8 08', 'WALLOWA-9': 'WALLOWA-LOSTINE #9 09', 'WALLOWA-10': 'WALLOWA-IMNAHA #10 10', 'WALLOWA-11': 'WALLOWA-FLORA #11 11', 'WALLOWA-12': 'WALLOWA-TROY #12 12', 'WASCO-PRECINCT 01': 'WASCO-WEST THE DALLES 01', 'WASCO-PRECINCT 02': 'WASCO-COLONEL WRIGHT 02', 'WASCO-PRECINCT 03': 'WASCO-MID THE DALLES 03', 'WASCO-PRECINCT 04': 'WASCO-EAST THE DALLES 04', 'WASCO-PRECINCT 05': 'WASCO-ROWENA-MOSIER 05', 'WASCO-PRECINCT 06': 'WASCO-CHENOWITH AREA 06', 'WASCO-PRECINCT 07': 'WASCO-CHERRY HTS/MILL CREEK/DRY HOLLOW 07', 'WASCO-PRECINCT 08': 'WASCO-PETERSBURG-DUFUR 08', 'WASCO-PRECINCT 09': 'WASCO-TYGH VALLEY-WAMIC 09', 'WASCO-PRECINCT 10': 'WASCO-MAUPIN CITY/W MAUPIN/WARM SPRINGS 10', 'WASCO-PRECINCT 11': 'WASCO-RURAL TD 11', 'WASCO-PRECINCT 12': 'WASCO-DUFUR & PETERSBURG 12', 'WASCO-PRECINCT 13': 'WASCO-TYGH VALLEY 13', 'WASCO-PRECINCT 14': 'WASCO-ANTELOPE/SHANIKO/E MAUPIN 14', 'WASHINGTON-PRECINCT 301': 'WASHINGTON-301 301', 'WASHINGTON-PRECINCT 302': 'WASHINGTON-302 302', 'WASHINGTON-PRECINCT 303': 'WASHINGTON-303 303', 'WASHINGTON-PRECINCT 304': 'WASHINGTON-304 304', 'WASHINGTON-PRECINCT 305': 'WASHINGTON-305 305', 'WASHINGTON-PRECINCT 306': 'WASHINGTON-306 306', 'WASHINGTON-PRECINCT 307': 'WASHINGTON-307 307', 'WASHINGTON-PRECINCT 308': 'WASHINGTON-308 308', 'WASHINGTON-PRECINCT 309': 'WASHINGTON-309 309', 'WASHINGTON-PRECINCT 310': 'WASHINGTON-310 310', 'WASHINGTON-PRECINCT 311': 'WASHINGTON-311 311', 'WASHINGTON-PRECINCT 312': 'WASHINGTON-312 312', 'WASHINGTON-PRECINCT 313': 'WASHINGTON-313 313', 'WASHINGTON-PRECINCT 314': 'WASHINGTON-314 314', 'WASHINGTON-PRECINCT 315': 'WASHINGTON-315 315', 'WASHINGTON-PRECINCT 316': 'WASHINGTON-316 316', 'WASHINGTON-PRECINCT 317': 'WASHINGTON-317 317', 'WASHINGTON-PRECINCT 318': 'WASHINGTON-318 318', 'WASHINGTON-PRECINCT 319': 'WASHINGTON-319 319', 'WASHINGTON-PRECINCT 320': 'WASHINGTON-320 320', 'WASHINGTON-PRECINCT 321': 'WASHINGTON-321 321', 'WASHINGTON-PRECINCT 322': 'WASHINGTON-322 322', 'WASHINGTON-PRECINCT 323': 'WASHINGTON-323 323', 'WASHINGTON-PRECINCT 324': 'WASHINGTON-324 324', 'WASHINGTON-PRECINCT 325': 'WASHINGTON-325 325', 'WASHINGTON-PRECINCT 326': 'WASHINGTON-326 326', 'WASHINGTON-PRECINCT 327': 'WASHINGTON-327 327', 'WASHINGTON-PRECINCT 328': 'WASHINGTON-328 328', 'WASHINGTON-PRECINCT 329': 'WASHINGTON-329 329', 'WASHINGTON-PRECINCT 330': 'WASHINGTON-330 330', 'WASHINGTON-PRECINCT 331': 'WASHINGTON-331 331', 'WASHINGTON-PRECINCT 332': 'WASHINGTON-332 332', 'WASHINGTON-PRECINCT 333': 'WASHINGTON-333 333', 'WASHINGTON-PRECINCT 334': 'WASHINGTON-334 334', 'WASHINGTON-PRECINCT 335': 'WASHINGTON-335 335', 'WASHINGTON-PRECINCT 336': 'WASHINGTON-336 336', 'WASHINGTON-PRECINCT 337': 'WASHINGTON-337 337', 'WASHINGTON-PRECINCT 338': 'WASHINGTON-338 338', 'WASHINGTON-PRECINCT 339': 'WASHINGTON-339 339', 'WASHINGTON-PRECINCT 340': 'WASHINGTON-340 340', 'WASHINGTON-PRECINCT 341': 'WASHINGTON-341 341', 'WASHINGTON-PRECINCT 342': 'WASHINGTON-342 342', 'WASHINGTON-PRECINCT 343': 'WASHINGTON-343 343', 'WASHINGTON-PRECINCT 344': 'WASHINGTON-344 344', 'WASHINGTON-PRECINCT 345': 'WASHINGTON-345 345', 'WASHINGTON-PRECINCT 346': 'WASHINGTON-346 346', 'WASHINGTON-PRECINCT 347': 'WASHINGTON-347 347', 'WASHINGTON-PRECINCT 348': 'WASHINGTON-348 348', 'WASHINGTON-PRECINCT 349': 'WASHINGTON-349 349', 'WASHINGTON-PRECINCT 350': 'WASHINGTON-350 350', 'WASHINGTON-PRECINCT 351': 'WASHINGTON-351 351', 'WASHINGTON-PRECINCT 352': 'WASHINGTON-352 352', 'WASHINGTON-PRECINCT 353': 'WASHINGTON-353 353', 'WASHINGTON-PRECINCT 354': 'WASHINGTON-354 354', 'WASHINGTON-PRECINCT 355': 'WASHINGTON-355 355', 'WASHINGTON-PRECINCT 356': 'WASHINGTON-356 356', 'WASHINGTON-PRECINCT 357': 'WASHINGTON-357 357', 'WASHINGTON-PRECINCT 358': 'WASHINGTON-358 358', 'WASHINGTON-PRECINCT 359': 'WASHINGTON-359 359', 'WASHINGTON-PRECINCT 360': 'WASHINGTON-360 360', 'WASHINGTON-PRECINCT 361': 'WASHINGTON-361 361', 'WASHINGTON-PRECINCT 362': 'WASHINGTON-362 362', 'WASHINGTON-PRECINCT 363': 'WASHINGTON-363 363', 'WASHINGTON-PRECINCT 364': 'WASHINGTON-364 364', 'WASHINGTON-PRECINCT 365': 'WASHINGTON-365 365', 'WASHINGTON-PRECINCT 366': 'WASHINGTON-366 366', 'WASHINGTON-PRECINCT 367': 'WASHINGTON-367 367', 'WASHINGTON-PRECINCT 368': 'WASHINGTON-368 368', 'WASHINGTON-PRECINCT 369': 'WASHINGTON-369 369', 'WASHINGTON-PRECINCT 370': 'WASHINGTON-370 370', 'WASHINGTON-PRECINCT 371': 'WASHINGTON-371 371', 'WASHINGTON-PRECINCT 372': 'WASHINGTON-372 372', 'WASHINGTON-PRECINCT 373': 'WASHINGTON-373 373', 'WASHINGTON-PRECINCT 374': 'WASHINGTON-374 374', 'WASHINGTON-PRECINCT 375': 'WASHINGTON-375 375', 'WASHINGTON-PRECINCT 376': 'WASHINGTON-376 376', 'WASHINGTON-PRECINCT 377': 'WASHINGTON-377 377', 'WASHINGTON-PRECINCT 378': 'WASHINGTON-378 378', 'WASHINGTON-PRECINCT 379': 'WASHINGTON-379 379', 'WASHINGTON-PRECINCT 380': 'WASHINGTON-380 380', 'WASHINGTON-PRECINCT 381': 'WASHINGTON-381 381', 'WASHINGTON-PRECINCT 382': 'WASHINGTON-382 382', 'WASHINGTON-PRECINCT 383': 'WASHINGTON-383 383', 'WASHINGTON-PRECINCT 384': 'WASHINGTON-384 384', 'WASHINGTON-PRECINCT 385': 'WASHINGTON-385 385', 'WASHINGTON-PRECINCT 386': 'WASHINGTON-386 386', 'WASHINGTON-PRECINCT 387': 'WASHINGTON-387 387', 'WASHINGTON-PRECINCT 388': 'WASHINGTON-388 388', 'WASHINGTON-PRECINCT 389': 'WASHINGTON-389 389', 'WASHINGTON-PRECINCT 390': 'WASHINGTON-390 390', 'WASHINGTON-PRECINCT 391': 'WASHINGTON-391 391', 'WASHINGTON-PRECINCT 392': 'WASHINGTON-392 392', 'WASHINGTON-PRECINCT 393': 'WASHINGTON-393 393', 'WASHINGTON-PRECINCT 394': 'WASHINGTON-394 394', 'WASHINGTON-PRECINCT 395': 'WASHINGTON-395 395', 'WASHINGTON-PRECINCT 396': 'WASHINGTON-396 396', 'WASHINGTON-PRECINCT 397': 'WASHINGTON-397 397', 'WASHINGTON-PRECINCT 398': 'WASHINGTON-398 398', 'WASHINGTON-PRECINCT 399': 'WASHINGTON-399 399', 'WASHINGTON-PRECINCT 400': 'WASHINGTON-400 400', 'WASHINGTON-PRECINCT 401': 'WASHINGTON-401 401', 'WASHINGTON-PRECINCT 402': 'WASHINGTON-402 402', 'WASHINGTON-PRECINCT 403': 'WASHINGTON-403 403', 'WASHINGTON-PRECINCT 404': 'WASHINGTON-404 404', 'WASHINGTON-PRECINCT 405': 'WASHINGTON-405 405', 'WASHINGTON-PRECINCT 406': 'WASHINGTON-406 406', 'WASHINGTON-PRECINCT 407': 'WASHINGTON-407 407', 'WASHINGTON-PRECINCT 408': 'WASHINGTON-408 408', 'WASHINGTON-PRECINCT 409': 'WASHINGTON-409 409', 'WASHINGTON-PRECINCT 410': 'WASHINGTON-410 410', 'WASHINGTON-PRECINCT 411': 'WASHINGTON-411 411', 'WASHINGTON-PRECINCT 412': 'WASHINGTON-412 412', 'WASHINGTON-PRECINCT 413': 'WASHINGTON-413 413', 'WASHINGTON-PRECINCT 414': 'WASHINGTON-414 414', 'WASHINGTON-PRECINCT 415': 'WASHINGTON-415 415', 'WASHINGTON-PRECINCT 416': 'WASHINGTON-416 416', 'WASHINGTON-PRECINCT 417': 'WASHINGTON-417 417', 'WASHINGTON-PRECINCT 418': 'WASHINGTON-418 418', 'WASHINGTON-PRECINCT 419': 'WASHINGTON-419 419', 'WASHINGTON-PRECINCT 420': 'WASHINGTON-420 420', 'WASHINGTON-PRECINCT 421': 'WASHINGTON-421 421', 'WASHINGTON-PRECINCT 422': 'WASHINGTON-422 422', 'WASHINGTON-PRECINCT 423': 'WASHINGTON-423 423', 'WASHINGTON-PRECINCT 424': 'WASHINGTON-424 424', 'WASHINGTON-PRECINCT 425': 'WASHINGTON-425 425', 'WASHINGTON-PRECINCT 426': 'WASHINGTON-426 426', 'WASHINGTON-PRECINCT 427': 'WASHINGTON-427 427', 'WASHINGTON-PRECINCT 428': 'WASHINGTON-428 428', 'WASHINGTON-PRECINCT 429': 'WASHINGTON-429 429', 'WASHINGTON-PRECINCT 430': 'WASHINGTON-430 430', 'WASHINGTON-PRECINCT 431': 'WASHINGTON-431 431', 'WASHINGTON-PRECINCT 432': 'WASHINGTON-432 432', 'WASHINGTON-PRECINCT 433': 'WASHINGTON-433 433', 'WASHINGTON-PRECINCT 434': 'WASHINGTON-434 434', 'WASHINGTON-PRECINCT 435': 'WASHINGTON-435 435', 'WASHINGTON-PRECINCT 436': 'WASHINGTON-436 436', 'WASHINGTON-PRECINCT 437': 'WASHINGTON-437 437', 'WASHINGTON-PRECINCT 438': 'WASHINGTON-438 438', 'WASHINGTON-PRECINCT 439': 'WASHINGTON-439 439', 'WASHINGTON-PRECINCT 440': 'WASHINGTON-440 440', 'WASHINGTON-PRECINCT 441': 'WASHINGTON-441 441', 'WASHINGTON-PRECINCT 442': 'WASHINGTON-442 442', 'WASHINGTON-PRECINCT 443': 'WASHINGTON-443 443', 'WASHINGTON-PRECINCT 444': 'WASHINGTON-444 444', 'WASHINGTON-PRECINCT 445': 'WASHINGTON-445 445', 'WHEELER-FOSSIL #1': 'WHEELER-FOSSIL 01', 'WHEELER-MITCHELL #3': 'WHEELER-MITCHELL 03', 'WHEELER-SPRAY #4': 'WHEELER-SPRAY 04', 'YAMHILL-1': 'YAMHILL-NEWBERG RURAL N,S,E 01', 'YAMHILL-2': 'YAMHILL-NEWBERG CITY DISTRICTS 3&6 02', 'YAMHILL-3': 'YAMHILL-NEWBERG CITY DISTRICTS 4&5 03', 'YAMHILL-4': 'YAMHILL-NEWBERG CITY DISTRICTS 1&2 04', 'YAMHILL-7': 'YAMHILL-DUNDEE CITY & RURAL 07', 'YAMHILL-9': 'YAMHILL-NEWBERG RURAL W 09', 'YAMHILL-10': 'YAMHILL-DAYTON CITY & RURAL 10', 'YAMHILL-11': 'YAMHILL-GASTON AREA 11', 'YAMHILL-12': 'YAMHILL-AMITY CITY & RURAL 12', 'YAMHILL-13': 'YAMHILL-YAMHILL CITY & RURAL 13', 'YAMHILL-14': 'YAMHILL-MCMINNVILLE WARD 1 14', 'YAMHILL-15': 'YAMHILL-MCMINNVILLE WARD 1 15', 'YAMHILL-16': 'YAMHILL-MCMINNVILLE WARD 3 16', 'YAMHILL-17': 'YAMHILL-MCMINNVILLE CITY WARD 2 17', 'YAMHILL-18': 'YAMHILL-MCMINNVILLE CITY WARD 2 18', 'YAMHILL-19': 'YAMHILL-MCMINNVILLE WARD 3 19', 'YAMHILL-20': 'YAMHILL-MCMINNVILLE & SHERIDAN RURAL 20', 'YAMHILL-21': 'YAMHILL-CARLTON CITY & RURAL 21', 'YAMHILL-22': 'YAMHILL-LAFAYETTE CITY & RURAL 22', 'YAMHILL-26': 'YAMHILL-SHERIDAN CITY & RURAL 26', 'YAMHILL-27': 'YAMHILL-WILLAMINIA CITY & RURAL 27', 'YAMHILL-28': 'YAMHILL-GRAND RONDE AREA 28'}

In [31]:
pivoted_results["unique_ID"] = pivoted_results["unique_ID"].map(source_vest_id_changes_dict).fillna(pivoted_results["unique_ID"])

In [32]:
pivoted_results = pivoted_results.fillna(0)


In [33]:
join_attempt_two = pd.merge(vest_or_20,pivoted_results,on="unique_ID",how="outer",indicator=True)
join_attempt_two["_merge"].value_counts()

both          1326
left_only        5
right_only       0
Name: _merge, dtype: int64

In [34]:
join_attempt_two[join_attempt_two["_merge"]=="left_only"].to_csv("./vest_only.csv")
join_attempt_two[join_attempt_two["_merge"]=="right_only"].to_csv("./pivoted.csv")

In [35]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    """Checks a merged dataframe with two election results at the precinct level

    Args:
      merged_df: DataFrame with one set of election results joined to another
      column_list: List of races that there are votes for
      vest_on_left: Boolean specifying whether VEST data is on the left side of merged_df
      name_col: String of the column name to refer to precincts when a difference occurs
      print_level: Integer that specifies how large the vote difference in a precinct must be to be printed

    Returns:
      Nothing, only prints out an analysis
    """
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)

In [36]:
precinct_votes_check(join_attempt_two[join_attempt_two["_merge"]=="both"],data_columns,True,"unique_ID",print_level=0)

G20PRELJOR ...................................................BAKER-BAKER CITY 5 05 (V) ...22  (S)....2 (D):   20
G20PREGHAW .......................................................GRANT-JOHN DAY 03 (V) ....8  (S)....0 (D):    8

There are  1326  total rows
2  of these rows have election result differences
1324  of these rows are the same

The max difference between any one shared column in a row is:  20.0
The average difference is:  14.0
There are  1 precinct results with a difference greater than 10

All precincts containing differences:
['BAKER-BAKER CITY 5 05', 'GRANT-JOHN DAY 03']


In [37]:
display(pivoted_results[pivoted_results["unique_ID"]=="COOS-BANDON CITY 16"])

cand_detailed,unique_ID,G20ATGDROS,G20ATGLHED,G20ATGOWRI,G20ATGRCRO,G20PREDBID,G20PREGHAW,G20PRELJOR,G20PREOWRI,G20PREPHUN,...,G20TREDREA,G20TREIHEN,G20TREOWRI,G20TRERGUD,G20USSDMER,G20USSGTAH,G20USSLDYE,G20USSOWRI,G20USSRPER,COUNTY
231,COOS-BANDON CITY 16,1002.0,33.0,3.0,943.0,1049.0,10.0,21.0,11.0,2.0,...,946.0,61.0,3.0,919.0,1054.0,19.0,21.0,1.0,925.0,Coos


There are a few different categories of differences:
    
    1. BAKER-BAKER CITY 5 05, G20PRELJOR 20 more votes in VEST
    2. GRANT county, difference in presidential results